<h1>setup</h1>

In [1]:
!pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 1.2 MB/s eta 0:00:00a 0:00:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.8/603.8 kB 1.3 MB/s eta 0:00:00-:--:--

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [4]:
!pip install psycopg2-binary

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.4 MB/s eta 0:00:00a 0:00:01


<h1>uri</h1>

In [1]:
db_type = 'postgresql'
adaptor = 'psycopg2'
user = 'postgres'
pas = '123456'
host = '127.0.0.1'
port = '5432'
db_name ='nima'
uri = f'{db_type}+{adaptor}://{user}:{pas}@{host}:{port}/{db_name}'

<h1>create engine</h1> 

In [2]:
from sqlalchemy import create_engine
engine = create_engine(uri)
# engine = create_engine(uri, echo=True)

<h1>disposing engine</h1>

In [26]:
from sqlalchemy import create_engine
engine2 = create_engine(uri)

In [27]:
engine2.dispose()

<h1>creating database</h1>

In [31]:
from sqlalchemy import create_engine
# baiad az ghabl ie database dashte bashim ke ie uri bede be ma
autocommit_engine = create_engine(uri, isolation_level='AUTOCOMMIT')


from sqlalchemy import text

with autocommit_engine.connect() as connection:
    executable = text('CREATE DATABASE new_nima')
    connection.execute(executable)

autocommit_engine.dispose()
db_type = 'postgresql'
adaptor = 'psycopg2'
user = 'postgres'
pas = '123456'
host = '127.0.0.1'
port = '5432'
db_name ='new_nima'
new_uri = f'{db_type}+{adaptor}://{user}:{pas}@{host}:{port}/{db_name}'
new_engine = create_engine(new_uri)


<h1>dropping a database</h1>

In [3]:
autocommit_engine = create_engine(uri, isolation_level='AUTOCOMMIT')

In [4]:
from sqlalchemy import text
executable = text('DROP DATABASE new_nima')

In [6]:
# aval tuie pg admin disconnect kardam
from sqlalchemy.orm import Session
with Session(autocommit_engine) as session:
    session.execute(executable)

In [7]:
autocommit_engine.dispose()

<h1>data types in sql</h1>

<h1>"" vs '' in PostgreSQL</h1>

<h1>value with DATE data type</h1>

<h1>constraint in sql</h1>

<h1>create table</h1>

<h2>by text api</h2>

In [3]:
from sqlalchemy import text
with engine.connect() as connection:
    executable = text('CREATE TABLE IF NOT EXISTS text_table(id SERIAL PRIMARY KEY, content VARCHAR);')
    connection.execute(executable)
    connection.commit()

<h2>by core api</h2>

In [4]:
from sqlalchemy import MetaData
metadata = MetaData()

from sqlalchemy import Table, Column, Integer, String
Table(
    'core_table',
    metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('content', String)
)

metadata.create_all(engine)


<h2>by declaritive orm</h2>

In [7]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()

from sqlalchemy import Column, Integer, String

class DeclaritiveTable(Base):
    __tablename__ = 'declaritive_table'
    id = Column(Integer, primary_key=True, autoincrement=True)
    content = Column(String)

Base.metadata.create_all(engine)

<h2>by registery orm</h2>

In [8]:
from sqlalchemy.orm import registry
reg = registry()

from sqlalchemy import Column, Integer, String

@reg.mapped
class RegistryTable:
    __tablename__ = 'registery_table'
    id = Column(Integer, primary_key=True, autoincrement=True)
    content = Column(String)

reg.metadata.create_all(engine)

<h1>moving between table object to orm class</h1>

<h2>to orm class</h2>

In [15]:
from sqlalchemy import MetaData
metadata = MetaData()

from sqlalchemy import Table, Column, Integer, String
table_obj = Table(
    'new_table',
    metadata,
    Column('id', Integer, primary_key=True, autoincrement=True),
    Column('content', String)
)

<h3>via registry</h3>

In [18]:
from sqlalchemy.orm import registry
reg = registry(metadata = metadata)

@reg.mapped
class OrmClass1:
    __table__ = table_obj

reg.metadata.create_all(engine)

<h3>via declaritive base</h3>

In [21]:
from sqlalchemy.orm import declarative_base
Base = declarative_base(metadata = metadata)

class OrmClass2(Base):
    __table__ = table_obj
    
Base.metadata.create_all(engine)

<h2>to table object</h2>

In [ ]:
from sqlalchemy.orm import registry
reg2 = registry()

from sqlalchemy import Column, Integer, String
@reg2.mapped
class OrmClass3:
    __tablename__ = 'new_table_2'
    id = Column(Integer, primary_key= True, autoincrement= True)
    content = Column(String)

table_obj_2 = OrmClass3.__table__
table_obj_2.create(engine)
# using table_obj.create in case of already existing table create exception
# but metadata.create_all (and Base.metadata.create_all and registry.metadata.create_all) check before creating

<h1>list of tables in database</h1>

In [50]:
from sqlalchemy import inspect

inspector = inspect(engine)
table_lst_in_db = inspector.get_table_names()
table_lst_in_db

['text_table',
 'core_table',
 'new_table',
 'new_table_2',
 't3',
 'declaritive_table',
 'registery_table',
 'core_orm']

<h1>list of tables in metadata</h1>

In [62]:
from sqlalchemy import MetaData, Table, Column, Integer
from sqlalchemy.orm import registry

metadata = MetaData()
reg = registry(metadata= metadata)

Table('added_to_metadata_1',
      metadata,
      Column('id', Integer, primary_key = True)
     )

@reg.mapped
class OClass:
    __tablename__='added_to_metadata_2'
    id = Column(Integer, primary_key= True)

In [65]:
list(metadata.tables.keys())

['added_to_metadata_1', 'added_to_metadata_2']

<h1>table description</h1>

In [9]:
from sqlalchemy import inspect

inspector = inspect(engine)
table_lst = inspector.get_table_names()

In [10]:
table_lst

['core_table',
 'new_table',
 'new_table_2',
 'declaritive_table',
 'registery_table',
 'core_orm']

<h2>inspect.get_columns(table_name)</h2>

In [12]:
column_lst = inspector.get_columns('core_table')

In [13]:
column_lst

[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': "nextval('core_table_id_seq'::regclass)",
  'autoincrement': True,
  'comment': None},
 {'name': 'content',
  'type': VARCHAR(),
  'nullable': True,
  'default': None,
  'autoincrement': False,
  'comment': None}]

In [14]:
from sqlalchemy import MetaData, Table
metadata = MetaData()
core_table = Table('core_table', metadata, autoload_with=engine)

<h2>printing table_obj</h2>

In [21]:
core_table

Table('core_table', MetaData(), Column('id', INTEGER(), table=<core_table>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x7f0cbe4cb290>, for_update=False)), Column('content', VARCHAR(), table=<core_table>), schema=None)

<h2>using attr of members of table_obj.columns or .c</h2>

In [22]:
[(c.name, c.type) for c in core_table.columns]

[('id', INTEGER()), ('content', VARCHAR())]

In [23]:
[(c.name, c.type) for c in core_table.c]

[('id', INTEGER()), ('content', VARCHAR())]

<h1>table_obj.columns == table_obj.c</h1>

In [26]:
core_table.columns is core_table.c

True

<h1>accessing one column of a table_obj</h1>

In [24]:
core_table.c.id

Column('id', INTEGER(), table=<core_table>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x7f0cbe4cb290>, for_update=False))

In [25]:
core_table.columns.id

Column('id', INTEGER(), table=<core_table>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x7f0cbe4cb290>, for_update=False))

<h1>accessing table objects from metadata</h1>

In [66]:
table_obj1 = metadata.tables['added_to_metadata_1']

<h1>accessing table objects from orm class</h1>

In [67]:
table_obj2 = OClass.__table__

<h1>remove a table from metadata</h1>

In [68]:
list(metadata.tables.keys())

['added_to_metadata_1', 'added_to_metadata_2']

In [69]:
metadata.remove(table_obj1)

In [71]:
metadata.remove(table_obj2)

In [72]:
list(metadata.tables.keys())

[]

<h1>checking existance of a table</h1>

In [89]:
from sqlalchemy import inspect, Table, Column, Integer, String

inspector = inspect(engine)

if 'new_table' not in metadata.tables:
    if 'new_table' not in inspector.get_table_names():
        # Case 1: Not in metadata, not in DB → define and create
        table_obj = Table(
            'new_table',
            metadata,
            Column('id', Integer, primary_key=True),
            Column('content', String)
        )
        metadata.create_all(engine)
        print("Table created from scratch.")
    else:
        # Case 2: Not in metadata, but in DB → reflect
        table_obj = Table('new_table', metadata, autoload_with=engine)
        print("Table reflected from DB.")
else:
    # Case 3: Already in metadata → check DB status
    table_obj = metadata.tables['new_table']
    if 'new_table' not in inspector.get_table_names():
        metadata.create_all(engine)
        print("Metadata-defined table created in DB.")
    else:
        print("Table already exists in both metadata and DB. Reusing table object.")


Table reflected from DB.


<h1>reflecting table</h1>

<h2>via core</h2>

In [14]:
from sqlalchemy import MetaData

metadata = MetaData()
table_object = Table(
                    'core_table',
                    metadata,
                    autoload_with = engine)

<h1>reflecting all the tables at once</h1>

<h2>via core</h2>

In [11]:
from sqlalchemy import MetaData

metadata = MetaData()
metadata.reflect(bind=engine)
table_list = metadata.tables
table_list.keys()

dict_keys(['core_table', 'new_table', 'new_table_2', 'declaritive_table', 'registery_table', 'core_orm'])

<h2>via orm</h2>

In [19]:
from sqlalchemy.ext.automap import automap_base

Base = automap_base()
Base.prepare(autoload_with=engine)
TableClass1 = Base.classes.declaritive_table
TableClass2 = Base.classes.text_table


In [3]:
from sqlalchemy.ext.automap import automap_base

def prepare_base():
    Base = automap_base()
    Base.prepare(autoload_with=engine)
    return Base

<h1>modern orm</h1>

In [9]:
from sqlalchemy import create_engine
from sqlalchemy.orm import registry, declarative_base
from sqlalchemy.ext.automap import automap_base

# 1. Engine
engine = create_engine(uri)

# 2. Registry
mapper_registry = registry()

# 3. Declarative base using custom registry
Base = automap_base(declarative_base=mapper_registry.generate_base())

# 4. Prepare
Base.prepare(autoload_with=engine)


<h1>dropping a table</h1>

In [34]:
from sqlalchemy.orm import registry
mapper_registery = registry()

from sqlalchemy import Column, Integer

@mapper_registery.mapped
class OrmClass1:
    __tablename__ = 't1'
    id = Column(Integer, primary_key= True, autoincrement= True)

@mapper_registery.mapped
class OrmClass2:
    __tablename__ = 't2'
    id = Column(Integer, primary_key= True, autoincrement= True)

@mapper_registery.mapped
class OrmClass3:
    __tablename__ = 't3'
    id = Column(Integer, primary_key= True, autoincrement= True)

mapper_registery.metadata.create_all(engine)
    

<h2>via Table.drop</h2>

In [45]:
from sqlalchemy import MetaData, Table, inspect

inspector = inspect(engine)

table_lst = inspector.get_table_names()
if 't1' in table_lst:
    metadata = MetaData()
    reflected_t1 = Table('t1',
                         metadata,
                         autoload_with= engine)
    reflected_t1.drop(engine)


<h2>via text api</h2>

In [48]:
from sqlalchemy import text, inspect

inspector = inspect(engine)
table_lst = inspector.get_table_names()

if 't2' in table_lst:
    with engine.connect() as connection:
        executable = text('DROP TABLE IF EXISTS t2')
        connection.execute(executable)
        connection.commit()

<h1>adding tables from db to metadata</h1>

In [75]:
from sqlalchemy import inspect
inspector = inspect(engine)
table_lst_in_db = inspector.get_table_names()

In [76]:
table_lst_in_db

['text_table',
 'core_table',
 'new_table',
 'new_table_2',
 't3',
 'declaritive_table',
 'registery_table',
 'core_orm']

In [79]:
from sqlalchemy import MetaData
metadata = MetaData()
table_lst_in_metadata = list(metadata.tables.keys())

In [80]:
table_lst_in_metadata

[]

In [81]:
from sqlalchemy import Table
Table('t3', metadata, autoload_with=engine)
Table('text_table', metadata, autoload_with=engine)

Table('text_table', MetaData(), Column('id', INTEGER(), table=<text_table>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x7faf98217b00>, for_update=False)), Column('content', VARCHAR(), table=<text_table>), schema=None)

In [82]:
table_lst_in_metadata = list(metadata.tables.keys())

In [83]:
table_lst_in_metadata

['t3', 'text_table']

<h1>dropping several tables</h1>

In [84]:
metadata.drop_all(engine)

In [87]:
inspector = inspect(engine)
table_lst_in_db = inspector.get_table_names()

In [88]:
table_lst_in_db

['core_table',
 'new_table',
 'new_table_2',
 'declaritive_table',
 'registery_table',
 'core_orm']

<h1>insert data</h1>

In [5]:
from sqlalchemy import inspect
inspector = inspect(engine)

In [8]:
inspector.get_table_names()

['core_table',
 'new_table',
 'new_table_2',
 'declaritive_table',
 'registery_table',
 'core_orm']

<h2>via text</h2>

In [34]:
from sqlalchemy import text
from sqlalchemy.orm import Session

executable = text('INSERT INTO core_orm (content) VALUES (:content)')
values_dict = {'content':'noura'}

with Session(engine) as session:
    session.execute(executable, values_dict)
    session.commit()

In [11]:
from sqlalchemy import text

with engine.connect() as connection:
    executable = text('INSERT INTO core_orm (content) VALUES (:content)')
    connection.execute(executable, {'content':'nima'})
    connection.commit()

In [30]:
from sqlalchemy import text

with engine.connect() as connection:
    executable = text("INSERT INTO core_orm (content) VALUES ('nick')")
    connection.execute(executable)
    connection.commit()

In [32]:
from sqlalchemy import text

with engine.connect() as connection:
    executable = text("INSERT INTO core_orm (content) VALUES (noura)")
    connection.execute(executable)
    connection.commit()

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "noura" does not exist
LINE 1: INSERT INTO core_orm (content) VALUES (noura)
                                               ^

[SQL: INSERT INTO core_orm (content) VALUES (noura)]
(Background on this error at: https://sqlalche.me/e/20/f405)

<h2>via core</h2>

In [14]:
from sqlalchemy import Table, MetaData, insert

metadata = MetaData()
table_obj = Table('core_orm', metadata, autoload_with=engine)

with engine.connect() as connection:
    executable = insert(table_obj).values(content='banafsheh')
    connection.execute(executable)
    connection.commit()

<h2>via orm</h2>

In [17]:
from sqlalchemy.orm import registry, Session

reg = registry()

@reg.mapped
class OrmClass:
    __table__ = table_obj

row = OrmClass(content = 'asal')
with Session(engine) as session:
    session.add(row)
    session.commit()

<h1>select query</h1>

In [34]:
from sqlalchemy import select

<h2>via text</h2>

In [22]:
executable = text('SELECT * FROM core_orm')
with Session(engine) as session:
    result = session.execute(executable)
    row_lst = result.fetchall()
row_lst

[(1, 'nima'), (2, 'banafsheh'), (3, 'asal')]

<h2>via core</h2>

In [33]:
executable = select(table_obj)
with Session(engine) as session:
    result = session.execute(executable)
    row_lst = result.fetchall()
row_lst

[(1, 'nima'), (2, 'banafsheh'), (3, 'asal')]

<h2>via orm</h2>

In [30]:
executable = select(OrmClass)
with Session(engine) as session:
    result = session.scalars(executable)
    obj_lst = result.all()
[obj.content for obj in obj_lst]

['nima', 'banafsheh', 'asal']

<h1>select where query</h1>

In [10]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import text, select
from sqlalchemy.orm import Session

Base = automap_base()
Base.prepare(autoload_with=engine)
C = Base.classes.core_orm
C.__repr__ = lambda self:f'id={self.id}, content={self.content}'
t = C.__table__

def exec_stm (stm, value_dict=None):
    with Session(engine) as session:
        if value_dict:
            return print(session.execute(stm,value_dict).mappings().all())
        return print(session.execute(stm).mappings().all())


<h2>via text</h2>

In [15]:
stm = text("SELECT * FROM core_orm WHERE id = :value")
value_dict = {'value': 1}
exec_stm(stm,value_dict)

[{'id': 1, 'content': 'nima'}]


In [19]:
stm = text("SELECT * FROM core_orm WHERE id = :value1 OR content = :value2")
value_dict = {'value1': 1, 'value2' : 'banafsheh'}
exec_stm(stm,value_dict)

[{'id': 1, 'content': 'nima'}, {'id': 2, 'content': 'banafsheh'}, {'id': 7, 'content': 'banafsheh'}]


In [42]:
executable = text('SELECT content FROM core_orm WHERE id < 3')
with Session(engine) as session:
    result = session.execute(executable)
    row_lst = result.fetchall()
row_lst

[('nima',), ('banafsheh',)]

<h2>via core</h2>

In [16]:
stm = select(t).where(t.c.id == 1)
exec_stm(stm)

[{'id': 1, 'content': 'nima'}]


In [22]:
from sqlalchemy import or_
stm = select(t).where(or_(t.c.id == 1, t.c.content == 'banafsheh'))
exec_stm(stm)

[{'id': 1, 'content': 'nima'}, {'id': 2, 'content': 'banafsheh'}, {'id': 7, 'content': 'banafsheh'}]


In [43]:
executable = select(table_obj.c.content).where(table_obj.c.id < 3)
with Session(engine) as session:
    result = session.execute(executable)
    row_lst = result.fetchall()
row_lst

[('nima',), ('banafsheh',)]

<h2>via orm</h2>

In [18]:
stm = select(C).where(C.id == 1)
exec_stm(stm)

[{'core_orm': id=1, content=nima}]


In [23]:
stm = select(C).where(or_(C.id == 1, C.content == 'banafsheh'))
exec_stm(stm)

[{'core_orm': id=1, content=nima}, {'core_orm': id=2, content=banafsheh}, {'core_orm': id=7, content=banafsheh}]


In [44]:
executable = select(OrmClass.content).where(OrmClass.id < 3)
with Session(engine) as session:
    result = session.scalars(executable)
    obj_lst = result.all()
[obj for obj in obj_lst]

['nima', 'banafsheh']

<h1>print table by orm</h1> 

In [3]:
from sqlalchemy import MetaData, Table
from sqlalchemy.orm import registry

metadata = MetaData()
table = Table('core_orm', metadata, autoload_with=engine)

reg = registry(metadata=metadata)

@reg.mapped
class OClass:
    __table__ = table
    def __repr__(self):
        return f'id={self.id}, content={self.content}'
    

In [19]:
from sqlalchemy import Select
from sqlalchemy.orm import Session

def print_table():
    executable = Select(OClass)
    with Session(engine) as session:
        result = session.scalars(executable)
        obj_list = result.all()
    print(obj_list)

In [20]:
print_table()

[id=1, content=nima, id=2, content=banafsheh, id=3, content=asal, id=4, content=nick, id=5, content=noura]


<h1>order by</h1>

<h2>text</h2>

In [41]:
from sqlalchemy import text

executable = text("SELECT * from core_orm order by content")
with Session(engine) as session:
    result = session.execute(executable)
    row_list = result.all()
row_list

[(3, 'asal'), (2, 'banafsheh'), (4, 'nick'), (1, 'nima'), (5, 'noura')]

<h2>core</h2>

In [27]:
from sqlalchemy import select

executable = select(table).order_by(table.columns.content)
with Session(engine) as session:
    result = session.execute(executable)
    dict_result = session.execute(executable).mappings()
    row_list = result.all()
    dict_list = dict_result.all()
row_list

[(3, 'asal'), (2, 'banafsheh'), (4, 'nick'), (1, 'nima'), (5, 'noura')]

In [28]:
dict_list

[{'id': 3, 'content': 'asal'},
 {'id': 2, 'content': 'banafsheh'},
 {'id': 4, 'content': 'nick'},
 {'id': 1, 'content': 'nima'},
 {'id': 5, 'content': 'noura'}]

<h2>orm</h2>

In [45]:
from sqlalchemy import select

executable = select(OClass).order_by(OClass.content)
with Session(engine) as session:
    result = session.scalars(executable)
    obj_list = result.all()
obj_list

[id=3, content=asal,
 id=2, content=banafsheh,
 id=4, content=nick,
 id=1, content=nima,
 id=5, content=noura]

<h1>correct way of running an executable</h1>

In [1]:
def executer(stm):
    with Session(engine) as session:
        print(session.execute(stm).mappings().all())

<h1>order by descending</h1>

<h2>text</h2>

In [50]:
stm1 = text("SELECT * FROM core_orm ORDER BY content DESC")

In [51]:
executer(stm1)

[{'id': 5, 'content': 'noura'}, {'id': 1, 'content': 'nima'}, {'id': 4, 'content': 'nick'}, {'id': 2, 'content': 'banafsheh'}, {'id': 3, 'content': 'asal'}]


<h2>core</h2>

In [55]:
stm2 = select(table).order_by(table.columns.content.desc())

In [56]:
executer(stm2)

[{'id': 5, 'content': 'noura'}, {'id': 1, 'content': 'nima'}, {'id': 4, 'content': 'nick'}, {'id': 2, 'content': 'banafsheh'}, {'id': 3, 'content': 'asal'}]


In [57]:
stm3 = select(OClass).order_by(OClass.content.desc())

In [58]:
executer(stm3)

[{'OClass': id=5, content=noura}, {'OClass': id=1, content=nima}, {'OClass': id=4, content=nick}, {'OClass': id=2, content=banafsheh}, {'OClass': id=3, content=asal}]


<h1>order by several columns</h1>

<h1>distinct</h1>

In [4]:
Base = prepare_base()

In [7]:
T_class = Base.classes.core_orm

In [8]:
t_obj = T_class.__table__

In [11]:
from sqlalchemy import insert

stm = insert(t_obj).values(content = 'banafsheh')
new_row = T_class(content = 'nima')

In [15]:
from sqlalchemy.orm import Session

In [18]:
from sqlalchemy import select

def print_table():
    with Session(engine) as session:
        print(session.execute(select(t_obj)).mappings().all())

In [19]:
print_table()

[{'id': 1, 'content': 'nima'}, {'id': 2, 'content': 'banafsheh'}, {'id': 3, 'content': 'asal'}, {'id': 4, 'content': 'nick'}, {'id': 5, 'content': 'noura'}]


In [22]:
with Session(engine) as session:
    session.execute(stm)
    session.add(new_row)
    session.commit()

In [23]:
print_table()

[{'id': 1, 'content': 'nima'}, {'id': 2, 'content': 'banafsheh'}, {'id': 3, 'content': 'asal'}, {'id': 4, 'content': 'nick'}, {'id': 5, 'content': 'noura'}, {'id': 7, 'content': 'banafsheh'}, {'id': 8, 'content': 'nima'}]


In [26]:
def execute_stm(stm):
    with Session(engine) as session:
        result_lst = session.execute(stm).mappings().all()
        # session.commit() here it is not needed since select is readonly but no harm (i comment it out to not confuse)
        return (result_lst)

<h2>text</h2> 

In [27]:
from sqlalchemy import text
stm = text("SELECT DISTINCT content FROM core_orm")

In [28]:
print(execute_stm(stm))

[{'content': 'nick'}, {'content': 'noura'}, {'content': 'nima'}, {'content': 'asal'}, {'content': 'banafsheh'}]


<h2>core expr</h2> 

In [29]:
from sqlalchemy import select
stm = select(t_obj.c.content.distinct())

In [30]:
print(execute_stm(stm))

[{'content': 'nick'}, {'content': 'noura'}, {'content': 'nima'}, {'content': 'asal'}, {'content': 'banafsheh'}]


<h2>orm</h2> 

In [40]:
stm = select(T_class.content.distinct().label('content'))

In [42]:
print(execute_stm(stm))

[{'content': 'nick'}, {'content': 'noura'}, {'content': 'nima'}, {'content': 'asal'}, {'content': 'banafsheh'}]


<h1>select + bool expr</h1>

In [3]:
from sqlalchemy import select, text
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

Base = automap_base()
Base.prepare(autoload_with=engine)
C = Base.classes.core_orm
C.__repr__ = lambda self: f'id = {self.id}, content = {self.content}'
t = C.__table__

def exe(stm, dic=None):
    with Session(engine) as session:
        if dic:
            return print(session.execute(stm, dic).mappings().all())
        return print(session.execute(stm).mappings().all())    

<h2>text</h2>

In [34]:
exe(text("SELECT content <> :value AS result FROM core_orm"), {'value' : 'banafsheh'})

[{'result': True}, {'result': False}, {'result': True}, {'result': True}, {'result': True}, {'result': False}, {'result': True}]


<h2>core</h2>

In [32]:
bool_expr = t.c.content != 'banafsheh'
label = 'result'
select_stm = bool_expr.label(label)
full_stm = select(select_stm)
exe(full_stm)

[{'result': True}, {'result': False}, {'result': True}, {'result': True}, {'result': True}, {'result': False}, {'result': True}]


<h2>orm</h2>

In [31]:
bool_expr = C.content != 'banafsheh'
label = 'result'
select_stm = bool_expr.label(label)
full_stm = select(select_stm)
exe(full_stm)

[{'result': True}, {'result': False}, {'result': True}, {'result': True}, {'result': True}, {'result': False}, {'result': True}]


<h1>WHERE + IN</h1>

<h2>text</h2>

In [35]:
stm = text("SELECT * FROM core_orm WHERE content IN (:v1 , :v2)")
dic = {'v1' : 'nima', 'v2' : 'banafsheh'}
exe(stm, dic)

[{'id': 1, 'content': 'nima'}, {'id': 2, 'content': 'banafsheh'}, {'id': 7, 'content': 'banafsheh'}, {'id': 8, 'content': 'nima'}]


<h2>core</h2>

In [36]:
lst = ['nima', 'banafsheh']
stm = select(t).where(t.c.content.in_(lst))
exe(stm)

[{'id': 1, 'content': 'nima'}, {'id': 2, 'content': 'banafsheh'}, {'id': 7, 'content': 'banafsheh'}, {'id': 8, 'content': 'nima'}]


<h2>orm</h2>

In [37]:
lst = ['nima', 'banafsheh']
stm = select(C).where(C.content.in_(lst))
exe(stm)

[{'core_orm': id = 1, content = nima}, {'core_orm': id = 2, content = banafsheh}, {'core_orm': id = 7, content = banafsheh}, {'core_orm': id = 8, content = nima}]


<h1>WHERE BETWEEN AND</h1>

<h2>text</h2>

In [46]:
stm = text("SELECT * FROM core_orm WHERE id BETWEEN :v1 AND :v2")
dic = {'v1': 1, 'v2': 5}
exe(stm, dic)

[{'id': 1, 'content': 'nima'}, {'id': 2, 'content': 'banafsheh'}, {'id': 3, 'content': 'asal'}, {'id': 4, 'content': 'nick'}, {'id': 5, 'content': 'noura'}]


<h2>core</h2>

In [47]:
lst = [1, 5]
stm = select(t).where(t.c.id.between(*lst))
exe(stm)

[{'id': 1, 'content': 'nima'}, {'id': 2, 'content': 'banafsheh'}, {'id': 3, 'content': 'asal'}, {'id': 4, 'content': 'nick'}, {'id': 5, 'content': 'noura'}]


<h2>orm</h2>

In [48]:
stm = select(C).where(C.id.between(*lst))
exe(stm)

[{'core_orm': id = 1, content = nima}, {'core_orm': id = 2, content = banafsheh}, {'core_orm': id = 3, content = asal}, {'core_orm': id = 4, content = nick}, {'core_orm': id = 5, content = noura}]


<h1>WHERE NOT / where(~)</h1>

<h2>text</h2>

In [50]:
stm = text("SELECT * FROM core_orm WHERE NOT (id BETWEEN :v1 AND :v2)")
dic = {'v1': 1, 'v2': 5}
exe(stm, dic)

[{'id': 7, 'content': 'banafsheh'}, {'id': 8, 'content': 'nima'}]


<h2>core</h2>

In [52]:
lst = [1, 5]
stm = select(t).where(~t.c.id.between(*lst))
exe(stm)

[{'id': 7, 'content': 'banafsheh'}, {'id': 8, 'content': 'nima'}]


<h2>orm</h2>

In [53]:
stm = select(C).where(~C.id.between(*lst))
exe(stm)

[{'core_orm': id = 7, content = banafsheh}, {'core_orm': id = 8, content = nima}]


<h1>WHERE LIKE _</h1>

<h2>text</h2>

In [5]:
stm = text("SELECT * FROM core_orm WHERE content LIKE :pattern")
exe(stm, {'pattern': '____'})

[{'id': 1, 'content': 'nima'}, {'id': 3, 'content': 'asal'}, {'id': 4, 'content': 'nick'}, {'id': 8, 'content': 'nima'}]


<h2>core</h2>

In [9]:
stm = select(t).where(t.c.content.like('____'))
exe(stm)

[{'id': 1, 'content': 'nima'}, {'id': 3, 'content': 'asal'}, {'id': 4, 'content': 'nick'}, {'id': 8, 'content': 'nima'}]


<h2>orm</h2>

In [12]:
stm = select(C).where(C.content.like('____'))
exe(stm)

[{'core_orm': id = 1, content = nima}, {'core_orm': id = 3, content = asal}, {'core_orm': id = 4, content = nick}, {'core_orm': id = 8, content = nima}]


<h1>WHERE LIKE %</h1>

<h2>text</h2>

In [17]:
stm = text("SELECT * FROM core_orm WHERE content LIKE :pattern")
exe(stm, {'pattern': 'n%'})

[{'id': 1, 'content': 'nima'}, {'id': 4, 'content': 'nick'}, {'id': 5, 'content': 'noura'}, {'id': 8, 'content': 'nima'}]


<h2>core</h2>

In [15]:
stm = select(t).where(t.c.content.like('n%'))
exe(stm)

[{'id': 1, 'content': 'nima'}, {'id': 4, 'content': 'nick'}, {'id': 5, 'content': 'noura'}, {'id': 8, 'content': 'nima'}]


<h2>orm</h2>

In [16]:
stm = select(C).where(C.content.like('n%'))
exe(stm)

[{'core_orm': id = 1, content = nima}, {'core_orm': id = 4, content = nick}, {'core_orm': id = 5, content = noura}, {'core_orm': id = 8, content = nima}]


<h1>WHERE ILIKE %</h1>

<h2>text</h2>

In [18]:
stm = text("SELECT * FROM core_orm WHERE content ILIKE :pattern")
exe(stm, {'pattern': 'N%'})

[{'id': 1, 'content': 'nima'}, {'id': 4, 'content': 'nick'}, {'id': 5, 'content': 'noura'}, {'id': 8, 'content': 'nima'}]


<h2>core</h2>

In [21]:
stm = select(t).where(t.c.content.ilike('N%'))
exe(stm)

[{'id': 1, 'content': 'nima'}, {'id': 4, 'content': 'nick'}, {'id': 5, 'content': 'noura'}, {'id': 8, 'content': 'nima'}]


<h2>orm</h2>

In [19]:
stm = select(C).where(C.content.ilike('N%'))
exe(stm)

[{'core_orm': id = 1, content = nima}, {'core_orm': id = 4, content = nick}, {'core_orm': id = 5, content = noura}, {'core_orm': id = 8, content = nima}]


<h1> char vs varchar</h1>

<h1>int vs integer</h1>

<h1>:: cast</h1>

<h1>SQL Aggregate functions</h1>

<h1>GROUP BY syntax</h1>

<h1>COUNT(c1) vs COUNT(*)</h1>

<h1>COUNT(c1) GROUP BY</h1>

<h2>text</h2>

In [30]:
stm = text("SELECT content, COUNT(content) FROM core_orm GROUP BY content")
exe(stm)

[{'content': 'nick', 'count': 1}, {'content': 'noura', 'count': 1}, {'content': 'nima', 'count': 2}, {'content': 'asal', 'count': 1}, {'content': 'banafsheh', 'count': 2}]


<h2>core</h2>

In [31]:
from sqlalchemy import func

stm = select(t.c.content, func.count(t.c.content)).group_by(t.c.content)
exe(stm)

[{'content': 'nick', 'count_1': 1}, {'content': 'noura', 'count_1': 1}, {'content': 'nima', 'count_1': 2}, {'content': 'asal', 'count_1': 1}, {'content': 'banafsheh', 'count_1': 2}]


<h2>orm</h2>

In [32]:
stm = select(C.content, func.count(C.content)).group_by(C.content)
exe(stm)

[{'content': 'nick', 'count': 1}, {'content': 'noura', 'count': 1}, {'content': 'nima', 'count': 2}, {'content': 'asal', 'count': 1}, {'content': 'banafsheh', 'count': 2}]


<h1>COUNT(*) GROUP BY</h1>

<h2>text</h2>

In [33]:
stm = text("SELECT content, COUNT(*) FROM core_orm GROUP BY content")
exe(stm)

[{'content': 'nick', 'count': 1}, {'content': 'noura', 'count': 1}, {'content': 'nima', 'count': 2}, {'content': 'asal', 'count': 1}, {'content': 'banafsheh', 'count': 2}]


<h2>core</h2>

In [35]:
stm = select(t.c.content, func.count()).group_by(t.c.content)
exe(stm)

[{'content': 'nick', 'count_1': 1}, {'content': 'noura', 'count_1': 1}, {'content': 'nima', 'count_1': 2}, {'content': 'asal', 'count_1': 1}, {'content': 'banafsheh', 'count_1': 2}]


<h2>orm</h2>

In [34]:
stm = select(C.content, func.count()).group_by(C.content)
exe(stm)

[{'content': 'nick', 'count': 1}, {'content': 'noura', 'count': 1}, {'content': 'nima', 'count': 2}, {'content': 'asal', 'count': 1}, {'content': 'banafsheh', 'count': 2}]


<h1>STRING_AGG GROUP BY</h1>

In [55]:
def check_agg_type(stm):
    with Session(engine) as session:
        tuple_lst = session.execute(stm).all()
        sample_member = tuple_lst[0]
        agg_column = sample_member[1]
        return type(agg_column)

<h2>text</h2>

In [56]:
stm = text("SELECT content, STRING_AGG(id::text, ' , ') FROM core_orm GROUP BY content")
exe(stm)

[{'content': 'nick', 'string_agg': '4'}, {'content': 'noura', 'string_agg': '5'}, {'content': 'nima', 'string_agg': '1 , 8'}, {'content': 'asal', 'string_agg': '3'}, {'content': 'banafsheh', 'string_agg': '2 , 7'}]


In [58]:
check_agg_type(stm)

str

<h2>core</h2>

In [59]:
from sqlalchemy import cast, String

stm = select(t.c.content, func.string_agg(cast(t.c.id, String), text("' , '"))).group_by(t.c.content)
exe(stm)

[{'content': 'nick', 'string_agg_1': '4'}, {'content': 'noura', 'string_agg_1': '5'}, {'content': 'nima', 'string_agg_1': '1 , 8'}, {'content': 'asal', 'string_agg_1': '3'}, {'content': 'banafsheh', 'string_agg_1': '2 , 7'}]


In [60]:
check_agg_type(stm)

str

<h2>orm</h2>

In [62]:
stm = select(C.content, func.string_agg(cast(C.id, String), text("' , '"))).group_by(C.content)
exe(stm)

[{'content': 'nick', 'string_agg': '4'}, {'content': 'noura', 'string_agg': '5'}, {'content': 'nima', 'string_agg': '1 , 8'}, {'content': 'asal', 'string_agg': '3'}, {'content': 'banafsheh', 'string_agg': '2 , 7'}]


In [63]:
check_agg_type(stm)

str

<h1>ARRAY_AGG GROUP BY</h1>

<h2>text</h2>

In [64]:
stm = text("SELECT content, ARRAY_AGG(id) FROM core_orm GROUP BY content")
exe(stm)

[{'content': 'nick', 'array_agg': [4]}, {'content': 'noura', 'array_agg': [5]}, {'content': 'nima', 'array_agg': [1, 8]}, {'content': 'asal', 'array_agg': [3]}, {'content': 'banafsheh', 'array_agg': [2, 7]}]


In [65]:
check_agg_type(stm)

list

<h2>core</h2>

In [67]:
stm = select(t.c.content, func.array_agg(t.c.id)).group_by(t.c.content)
exe(stm)
check_agg_type(stm)

[{'content': 'nick', 'array_agg_1': [4]}, {'content': 'noura', 'array_agg_1': [5]}, {'content': 'nima', 'array_agg_1': [1, 8]}, {'content': 'asal', 'array_agg_1': [3]}, {'content': 'banafsheh', 'array_agg_1': [2, 7]}]


list

<h2>orm</h2>

In [68]:
stm = select(C.content, func.array_agg(C.id)).group_by(C.content)
exe(stm)
check_agg_type(stm)

[{'content': 'nick', 'array_agg': [4]}, {'content': 'noura', 'array_agg': [5]}, {'content': 'nima', 'array_agg': [1, 8]}, {'content': 'asal', 'array_agg': [3]}, {'content': 'banafsheh', 'array_agg': [2, 7]}]


list

<h1>HAVING</h1>

In [3]:
from sqlalchemy import create_engine
engine = create_engine(uri)

In [4]:
from sqlalchemy.ext.automap import automap_base
Base = automap_base()

In [5]:
Base.prepare(autoload_with=engine)

In [6]:
C = Base.classes.core_orm
C.__repr__ = lambda self:f'id={self.id}, content={self.content}'

In [7]:
t = C.__table__

In [8]:
from sqlalchemy.orm import Session

In [9]:
def exe(stm):
    with Session(engine) as session:
        print(session.execute(stm).mappings().all())

<h2>text</h2>

In [10]:
from sqlalchemy import text

In [87]:
st = """
SELECT content, COUNT(content), ARRAY_AGG(id), SUM(id)
FROM core_orm
WHERE id > 1 AND NOT (content LIKE '%ra')
GROUP BY content
HAVING SUM(id) > 3
order by content
"""

In [88]:
stm = text(st)
exe(stm)

[{'content': 'banafsheh', 'count': 2, 'array_agg': [2, 7], 'sum': 9}, {'content': 'nick', 'count': 1, 'array_agg': [4], 'sum': 4}, {'content': 'nima', 'count': 1, 'array_agg': [8], 'sum': 8}]


<h2>core</h2>

In [89]:
from sqlalchemy import select, func, and_

In [90]:
c = func.count
s = func.sum
a = func.array_agg 

content = t.c.content
id = t.c.id

cc = c(id).label('count(id)')
ss = s(id).label('sum(id)')
aa = a(id).label('array(id)')

stm = select(content, cc, aa, ss).where(and_(id > 1, ~content.like('%ra'))).group_by(content).having(ss > 3).order_by(content)

In [91]:
exe(stm)

[{'content': 'banafsheh', 'count(id)': 2, 'array(id)': [2, 7], 'sum(id)': 9}, {'content': 'nick', 'count(id)': 1, 'array(id)': [4], 'sum(id)': 4}, {'content': 'nima', 'count(id)': 1, 'array(id)': [8], 'sum(id)': 8}]


<h2>orm</h2>

In [80]:
content = C.content
id = C.id

In [81]:
stm = select(content, c(id), a(id), s(id)).where(and_(id > 1, ~content.like('%ra'))).group_by(content).having(s(id) > 3).order_by(content)

In [82]:
exe(stm)

[{'content': 'banafsheh', 'count': 2, 'array_agg': [2, 7], 'sum': 9}, {'content': 'nick', 'count': 1, 'array_agg': [4], 'sum': 4}, {'content': 'nima', 'count': 1, 'array_agg': [8], 'sum': 8}]


<h1>stm.compile</h1>

In [93]:
print(str(stm))

SELECT core_orm.content, count(core_orm.id) AS "count(id)", array_agg(core_orm.id) AS "array(id)", sum(core_orm.id) AS "sum(id)" 
FROM core_orm 
WHERE core_orm.id > :id_1 AND core_orm.content NOT LIKE :content_1 GROUP BY core_orm.content 
HAVING sum(core_orm.id) > :param_1 ORDER BY core_orm.content


In [95]:
print(stm.compile())

SELECT core_orm.content, count(core_orm.id) AS "count(id)", array_agg(core_orm.id) AS "array(id)", sum(core_orm.id) AS "sum(id)" 
FROM core_orm 
WHERE core_orm.id > :id_1 AND core_orm.content NOT LIKE :content_1 GROUP BY core_orm.content 
HAVING sum(core_orm.id) > :param_1 ORDER BY core_orm.content


In [99]:
print(stm.compile(compile_kwargs={'literal_binds': True}))

SELECT core_orm.content, count(core_orm.id) AS "count(id)", array_agg(core_orm.id) AS "array(id)", sum(core_orm.id) AS "sum(id)" 
FROM core_orm 
WHERE core_orm.id > 1 AND core_orm.content NOT LIKE '%ra' GROUP BY core_orm.content 
HAVING sum(core_orm.id) > 3 ORDER BY core_orm.content


<h1>aggregate functions without GROUP BY</h1>

<h2>text</h2>

In [12]:
st = """
SELECT SUM(id), AVG(id), ROUNd(AVG(id)), MAX(id), COUNT(id)
FROM core_orm
"""
stm = text(st)
exe(stm)

[{'sum': 30, 'avg': Decimal('4.2857142857142857'), 'round': Decimal('4'), 'max': 8, 'count': 7}]


<h2>core</h2>

In [16]:
from sqlalchemy import func, select

In [18]:
id = t.c.id
s = func.sum(id)
a = func.avg(id)
ra = func.round(func.avg(id))
m = func.max(id)
c = func.count(id)
stm = select(s, a, ra, m, c)
exe(stm)


[{'sum_1': 30, 'avg_1': Decimal('4.2857142857142857'), 'round_1': Decimal('4'), 'max_1': 8, 'count_1': 7}]


<h2>orm</h2>

In [19]:
id = C.id
s = func.sum(id)
a = func.avg(id)
ra = func.round(func.avg(id))
m = func.max(id)
c = func.count(id)
stm = select(s, a, ra, m, c)
exe(stm)

[{'sum': 30, 'avg': Decimal('4.2857142857142857'), 'round': Decimal('4'), 'max': 8, 'count': 7}]


<h1>literal vs literal_column</h1>

<h1>Numeric or Decimal</h1>

<h1>math operations without table</h1>

<h2>text</h2>

In [51]:
st = """
SELECT 
  (5 + 3) AS sum_result,
  (5/3) AS int_div,
  (5.0/3) AS float_div,
  FLOOR(5.0/3) AS floor_div,
  (5 % 3) AS remainder,
  MOD(5, 3) AS mod,
  ROUND(5.0/3) AS rounded_div;
"""
stm = text(st)
exe(stm)

[{'sum_result': 8, 'int_div': 1, 'float_div': Decimal('1.6666666666666667'), 'floor_div': Decimal('1'), 'remainder': 2, 'mod': 2, 'rounded_div': Decimal('2')}]


<h2>literal of python math operation + some func</h2>

In [87]:
from sqlalchemy import select, func, literal, cast, Numeric

o1 = literal(5 + 3).label('sum_result')
o2 = literal(5 // 3) .label('int_div')
o3 = literal(5.0 / 3).label('float_div')
o4 = cast(5.0 // 3, Numeric(1, 0 )).label('floor_div')
o4_v2 = func.floor(5.0 / 3).label('floor_div_2')
o5 = literal(5%3).label('remainder')
o6 = func.mod(5,3).label('mod')
o7 = func.round(5.0 / 3).label('rounded_div')


In [88]:
stm = select(o1, o2, o3, o4, o4_v2, o5, o6, o7)

In [89]:
exe(stm)

[{'sum_result': 8, 'int_div': 1, 'float_div': 1.6666666666666667, 'floor_div': Decimal('1'), 'floor_div_2': Decimal('1'), 'remainder': 2, 'mod': 2, 'rounded_div': Decimal('2')}]


<h2>literal_column of raw sql</h2>

In [91]:
from sqlalchemy import select, literal_column

o1 = literal_column('5 + 3').label('sum_result')
o2 = literal_column('5/3').label('int_div')
o3 = literal_column('5.0/3').label('float_div')
o4 = literal_column('FLOOR(5.0/3)').label('floor_div')
o5 = literal_column('5 % 3').label('remainder')
o6 = literal_column('MOD(5, 3)').label('mod')
o7 = literal_column('ROUND(5.0/3)').label('rounded_div')

stm = select(o1, o2, o3, o4, o4_v2, o5, o6, o7)
exe(stm)

[{'sum_result': 8, 'int_div': 1, 'float_div': Decimal('1.6666666666666667'), 'floor_div': Decimal('1'), 'floor_div_2': Decimal('1'), 'remainder': 2, 'mod': 2, 'rounded_div': Decimal('2')}]


<h1>ROUND(x,n)</h1>

In [11]:
exe(text("SELECT ROUND(3.12345,2)"))

[{'round': Decimal('3.12')}]


In [13]:
from sqlalchemy import literal, select,func

exe(select(func.round(literal(3.12345), 2)))

[{'round_1': Decimal('3.12')}]


<h1>math operations on columns</h1>

In [14]:
exe(text("SELECT 2 * id FROM core_orm"))

[{'?column?': 2}, {'?column?': 4}, {'?column?': 6}, {'?column?': 8}, {'?column?': 10}, {'?column?': 14}, {'?column?': 16}]


In [16]:
stm = select(literal(2) * C.id)
exe(stm)

[{'_no_label': 2}, {'_no_label': 4}, {'_no_label': 6}, {'_no_label': 8}, {'_no_label': 10}, {'_no_label': 14}, {'_no_label': 16}]


<h1>NULL vs None</h1>

<h1>is_ / IS</h1>

<h1>insert NULL</h1>

In [2]:
from sqlalchemy import create_engine

In [3]:
engine = create_engine(uri)

In [4]:
from sqlalchemy.ext.automap import automap_base

In [5]:
Base = automap_base()

In [7]:
Base.prepare(autoload_with = engine)

In [8]:
C = Base.classes.core_orm
C.__repr__= lambda self : f'id = {self.id} , content = {self.content}'

In [9]:
t = C.__table__

<h2>text</h2>

In [10]:
from sqlalchemy import text

In [11]:
stm = text('INSERT INTO core_orm(content) VALUES(NULL)')

In [13]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    session.execute(stm)
    session.commit()

<h2>core</h2>

In [14]:
from sqlalchemy import insert

stm = insert(t).values(content = None)
with Session(engine) as session:
    session.execute(stm)
    session.commit()

<h2>orm</h2>

In [15]:
row = C(content = None)
with Session(engine) as session:
    session.add(row)
    session.commit()

In [16]:
from sqlalchemy import select

In [18]:
with Session(engine) as session:
    r = session.execute(select(t)).mappings().all()

In [19]:
print(r)

[{'id': 1, 'content': 'nima'}, {'id': 2, 'content': 'banafsheh'}, {'id': 3, 'content': 'asal'}, {'id': 4, 'content': 'nick'}, {'id': 5, 'content': 'noura'}, {'id': 7, 'content': 'banafsheh'}, {'id': 8, 'content': 'nima'}, {'id': 9, 'content': None}, {'id': 10, 'content': None}, {'id': 11, 'content': None}]


<h1>check for NULLs</h1>

In [20]:
def exe(stm):
    with Session(engine) as session:
        print(session.execute(stm).mappings().all())

<h2>text</h2>

In [21]:
exe(text("SELECT * FROM core_orm WHERE content IS NULL"))

[{'id': 9, 'content': None}, {'id': 10, 'content': None}, {'id': 11, 'content': None}]


<h2>core</h2>

In [25]:
exe (select(C).where(C.content.is_(None)))

[{'core_orm': id = 9 , content = None}, {'core_orm': id = 10 , content = None}, {'core_orm': id = 11 , content = None}]


<h1>check for not NULLS</h1>

<h2>text</h2>

In [26]:
exe(text("SELECT * FROM core_orm WHERE content IS NOT NULL"))

[{'id': 1, 'content': 'nima'}, {'id': 2, 'content': 'banafsheh'}, {'id': 3, 'content': 'asal'}, {'id': 4, 'content': 'nick'}, {'id': 5, 'content': 'noura'}, {'id': 7, 'content': 'banafsheh'}, {'id': 8, 'content': 'nima'}]


<h2>core/orm</h2>

In [27]:
exe (select(C).where(C.content.is_not(None)))

[{'core_orm': id = 1 , content = nima}, {'core_orm': id = 2 , content = banafsheh}, {'core_orm': id = 3 , content = asal}, {'core_orm': id = 4 , content = nick}, {'core_orm': id = 5 , content = noura}, {'core_orm': id = 7 , content = banafsheh}, {'core_orm': id = 8 , content = nima}]


<h1>Conditional Structures</h1>

<h2>NULLIF</h2>

<h3>text</h3>

In [30]:
exe(text("SELECT content FROM core_orm"))

[{'content': 'nima'}, {'content': 'banafsheh'}, {'content': 'asal'}, {'content': 'nick'}, {'content': 'noura'}, {'content': 'banafsheh'}, {'content': 'nima'}, {'content': None}, {'content': None}, {'content': None}]


In [28]:
exe(text("SELECT NULLIF(content, 'nick') FROM core_orm"))

[{'nullif': 'nima'}, {'nullif': 'banafsheh'}, {'nullif': 'asal'}, {'nullif': None}, {'nullif': 'noura'}, {'nullif': 'banafsheh'}, {'nullif': 'nima'}, {'nullif': None}, {'nullif': None}, {'nullif': None}]


<h3>core/orm</h3>

In [38]:
from sqlalchemy import func
nullif_nick = func.nullif(C.content, 'nick')
exe(select(nullif_nick))

[{'nullif': 'nima'}, {'nullif': 'banafsheh'}, {'nullif': 'asal'}, {'nullif': None}, {'nullif': 'noura'}, {'nullif': 'banafsheh'}, {'nullif': 'nima'}, {'nullif': None}, {'nullif': None}, {'nullif': None}]


<h2>COALESCE</h2>

<h3>text</h3>

In [39]:
exe(text("SELECT COALESCE(content, 'no content') FROM core_orm"))

[{'coalesce': 'nima'}, {'coalesce': 'banafsheh'}, {'coalesce': 'asal'}, {'coalesce': 'nick'}, {'coalesce': 'noura'}, {'coalesce': 'banafsheh'}, {'coalesce': 'nima'}, {'coalesce': 'no content'}, {'coalesce': 'no content'}, {'coalesce': 'no content'}]


<h3>core/orm</h3>

In [40]:
null_to_no_contetn = func.coalesce(C.content, 'no content')
exe (select(null_to_no_contetn))

[{'coalesce': 'nima'}, {'coalesce': 'banafsheh'}, {'coalesce': 'asal'}, {'coalesce': 'nick'}, {'coalesce': 'noura'}, {'coalesce': 'banafsheh'}, {'coalesce': 'nima'}, {'coalesce': 'no content'}, {'coalesce': 'no content'}, {'coalesce': 'no content'}]


<h2>SIMPLE CASE</h2>

<h3>text</h3>

In [44]:
exe(text("""
SELECT
CASE content
WHEN 'nick' THEN NULL
ELSE content
END
FROM core_orm
"""))

[{'content': 'nima'}, {'content': 'banafsheh'}, {'content': 'asal'}, {'content': None}, {'content': 'noura'}, {'content': 'banafsheh'}, {'content': 'nima'}, {'content': None}, {'content': None}, {'content': None}]


<h3>core/orm</h3>

<h2>SEARCHED CASE</h2>

<h3>text</h3>

In [48]:
exe(text("""
SELECT
CASE
WHEN content IS NULL THEN 'no content'
WHEN content = 'nick' THEN NULL
ELSE content
END
FROM core_orm
"""))

[{'content': 'nima'}, {'content': 'banafsheh'}, {'content': 'asal'}, {'content': None}, {'content': 'noura'}, {'content': 'banafsheh'}, {'content': 'nima'}, {'content': 'no content'}, {'content': 'no content'}, {'content': 'no content'}]


<h3>core/orm</h3>

In [55]:
from sqlalchemy import case

In [57]:
c = case(
    (C.content.is_(None), 'no content'),
    (C.content == 'nick', None),
    else_ = C.content
)
exe(select(c))

[{'_no_label': 'nima'}, {'_no_label': 'banafsheh'}, {'_no_label': 'asal'}, {'_no_label': None}, {'_no_label': 'noura'}, {'_no_label': 'banafsheh'}, {'_no_label': 'nima'}, {'_no_label': 'no content'}, {'_no_label': 'no content'}, {'_no_label': 'no content'}]


<h1>NULLIF for preventing devide by zero</h1>

<h3>text</h3>

In [49]:
exe(text("SELECT 10/NULLIF(0, 0)"))

[{'?column?': None}]


<h3>core/orm</h3>

In [50]:
from sqlalchemy import literal

In [52]:
exe(select(literal(10) / func.nullif(0,0)))

[{'anon_1': None}]


<h1>4 tools of registry</h1>

In [1]:
from sqlalchemy.orm import registry
sqlalchemy_resigtry = registry()

<h2>metadata</h2>

In [2]:
# 1
metadata = sqlalchemy_resigtry.metadata

<h2>generate_base</h2>

In [3]:
# 2
Base = sqlalchemy_resigtry.generate_base()

<h2>mapped decorator</h2>

In [ ]:
# 3 Decorator
mapped = sqlalchemy_resigtry.mapped

<h2>bind class to table obj</h2>

In [6]:
# 4
bind_class_to_table = sqlalchemy_resigtry.map_imperatively

<h1>Defenitions</h1>

<h1>Altering a table</h1>

In [5]:
from sqlalchemy import text
raw = """
ALTER TABLE date
ADD COLUMN
with_zone TIMESTAMPTZ
"""
stm = text(raw)

In [6]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    session.execute(stm)
    session.commit()

In [14]:
from sqlalchemy import text
raw = """
ALTER TABLE date
ADD COLUMN
time_zone TIMETZ
"""
stm = text(raw)

In [15]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    session.execute(stm)
    session.commit()

<h1>create a time related table</h1>

<h2>registry</h2>

In [4]:
from sqlalchemy import create_engine
engine = create_engine(uri)

from sqlalchemy.orm import registry
sqlalchemy_registry = registry()
metadata = sqlalchemy_registry.metadata
mapped = sqlalchemy_registry.mapped

from sqlalchemy import Column, Integer, String, Date, Time, TIMESTAMP

@mapped
class CDate:
    __tablename__ = 'tdate_orm'
    id = Column (Integer, primary_key= True, autoincrement= True)
    content = Column (String)
    timestamp_with_tz = Column (TIMESTAMP(timezone= True))
    timestamp_without_tz = Column (TIMESTAMP(timezone= False))
    time_with_tz = Column (Time(timezone= True))
    time_without_tz = Column (Time(timezone= False))
    date_column = Column (Date)

    @classmethod
    def describe(cls):
        col_lst = cls.__table__.columns
        description = {col.name : col.type for col in col_lst}
        return description

    def __repr__(self):
        col_lst = self.__class__.__table__.columns
        row = [(col.name, getattr(self, col.name)) for col in col_lst]
        class_name = self.__class__.__name__
        table_name = self.__class__.__tablename__
        return f"Instance of class {class_name} (table {table_name}):\n {row}"

                                   
metadata.create_all(engine)    

<h2>text</h2>

In [5]:
raw = """
CREATE TABLE IF NOT EXISTS tdate_text (
    id SERIAL PRIMARY KEY,
    content VARCHAR,
    timestamp_with_tz TIMESTAMPTZ,
    timestamp_without_tz TIMESTAMP,
    time_with_tz TIMETZ,
    time_without_tz TIME,
    date_column DATE
)
"""
from sqlalchemy import text
stm = text(raw)

from sqlalchemy.orm import Session
with Session(engine) as session:
    session.execute(stm)
    session.commit()

<h1>using enhancment mixin</h1>

In [36]:
from sqlalchemy import create_engine
engine = create_engine(uri)

from sqlalchemy.orm import registry
sqlalchemy_registry = registry()

from sqlalchemy.ext.automap import automap_base
Base = automap_base(declarative_base= sqlalchemy_registry.generate_base())
Base.prepare(autoload_with= engine)

from sqlalchemy import select, text
from sqlalchemy.orm import Session
class EnhancementMixin():
    @classmethod
    def describe(cls):
        col_lst = cls.__table__.columns
        description = {col.name : col.type for col in col_lst}
        return description

    def __repr__(self):
        col_lst = self.__class__.__table__.columns
        row = [(col.name, getattr(self, col.name)) for col in col_lst]
        class_name = self.__class__.__name__
        table_name = self.__class__.__tablename__
        return f"Instance of class {class_name} (table {table_name}):\n {row}"

    @classmethod
    def print_table(cls):
        table = cls.__table__
        with Session(engine) as session:
            print(session.execute(select(table)).mappings().all())

    @staticmethod
    def exe(stm):
        if isinstance(stm, str):
            stm = text(stm)
        with Session(engine) as session:
            session.execute(stm)
            session.commit()
        


ReflectedClass = Base.classes.tdate_orm
more_attr = {}
C = type(f'Enhanced{ReflectedClass.__name__}', (ReflectedClass, EnhancementMixin), more_attr)
# C ra nemishavad baraie sakhtan obj row estefade kard. az raveshe bad estefade kone
t = C.__table__




In [25]:
C.describe()

{'id': INTEGER(),
 'content': VARCHAR(),
 'timestamp_with_tz': TIMESTAMP(timezone=True),
 'timestamp_without_tz': TIMESTAMP(),
 'time_with_tz': TIME(timezone=True),
 'time_without_tz': TIME(),
 'date_column': DATE()}

<h1>using enhancment setattr with inheritance</h1>

In [27]:
class EnhancedClass(ReflectedClass):
    @classmethod
    def describe(cls):
        col_lst = cls.__table__.columns
        description = {col.name : col.type for col in col_lst}
        return description

    def __repr__(self):
        col_lst = self.__class__.__table__.columns
        row = [(col.name, getattr(self, col.name)) for col in col_lst]
        class_name = self.__class__.__name__
        table_name = self.__class__.__tablename__
        return f"Instance of class {class_name} (table {table_name}):\n {row}"

    @classmethod
    def print_table(cls):
        table = cls.__table__
        with Session(engine) as session:
            print(session.execute(select(table)).mappings().all())

    @staticmethod
    def exe(stm):
        if isinstance(stm, str):
            stm = text(stm)
        with Session(engine) as session:
            session.execute(stm)
            session.commit()


for attr in dir(EnhancedClass):
    if not attr.startswith('__'):
        setattr(ReflectedClass, attr, getattr(EnhancedClass, attr))

# it is too long do the next one

In [28]:
ReflectedClass.print_table()

[{'id': 1, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 12, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'timestamp_without_tz': datetime.datetime(2025, 5, 26, 12, 32, 15, 468258), 'time_with_tz': datetime.time(12, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'time_without_tz': datetime.time(12, 32, 15, 468258), 'date_column': datetime.date(2025, 5, 26)}, {'id': 2, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 13, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'timestamp_without_tz': datetime.datetime(2025, 5, 26, 13, 4, 17, 280505), 'time_with_tz': datetime.time(13, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'time_without_tz': datetime.time(13, 4, 17, 280505), 'date_column': datetime.date(2025, 5, 26)}, {'id': 3, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 13, 8, 4, 401263, tzinfo=datet

<h1>using enhancment setattr with functions</h1>

In [37]:
from sqlalchemy import select, text
from sqlalchemy.orm import Session

@classmethod
def describe(cls):
    col_lst = cls.__table__.columns
    return {col.name: col.type for col in col_lst}

@classmethod
def print_table(cls):
    with Session(engine) as session:
        print(session.execute(select(cls.__table__)).mappings().all())

@staticmethod
def exe(stm):
    if isinstance(stm, str):
        stm = text(stm)
    with Session(engine) as session:
        session.execute(stm)
        session.commit()

def repr_func(self):
    col_lst = self.__class__.__table__.columns
    row = [(col.name, getattr(self, col.name)) for col in col_lst]
    return f"Instance of class {self.__class__.__name__} (table {self.__class__.__tablename__}):\n {row}"

setattr(ReflectedClass, 'describe', describe)
setattr(ReflectedClass, 'print_table', print_table)
setattr(ReflectedClass, 'exe', exe)
setattr(ReflectedClass, '__repr__', repr_func)



In [38]:
ReflectedClass.describe()

{'id': INTEGER(),
 'content': VARCHAR(),
 'timestamp_with_tz': TIMESTAMP(timezone=True),
 'timestamp_without_tz': TIMESTAMP(),
 'time_with_tz': TIME(timezone=True),
 'time_without_tz': TIME(),
 'date_column': DATE()}

<h1>adding row to the time related table</h1>

<h2>text</h2>

In [5]:
raw = """
INSERT INTO tdate_orm
(
    timestamp_with_tz,
    timestamp_without_tz,
    time_with_tz,
    time_without_tz,
    date_column)
VALUES
( 
    NOW()::TIMESTAMPTZ, 
    NOW()::TIMESTAMP,
    NOW()::TIMETZ,
    NOW()::TIME,
    NOW()::DATE
)

"""
C.exe(raw)
C.print_table()

[{'id': 1, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 12, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'timestamp_without_tz': datetime.datetime(2025, 5, 26, 12, 32, 15, 468258), 'time_with_tz': datetime.time(12, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'time_without_tz': datetime.time(12, 32, 15, 468258), 'date_column': datetime.date(2025, 5, 26)}]


<h2>text and python</h2>

In [6]:
from datetime import datetime
from zoneinfo import ZoneInfo

timestamptz = datetime.now(ZoneInfo("Europe/Helsinki"))
timestamp = timestamptz.replace(tzinfo=None)

timetz = timestamptz.timetz()
time_ = timetz.replace(tzinfo=None)

date_ = timestamptz.date()

raw = """
INSERT INTO tdate_orm
(
    timestamp_with_tz,
    timestamp_without_tz,
    time_with_tz,
    time_without_tz,
    date_column)
VALUES
( 
    :v1, :v2, :v3, :v4, :v5
)
"""

from sqlalchemy import text
stm = text(raw).bindparams(
    v1 = timestamptz,
    v2 = timestamp,
    v3 = timetz,
    v4 = time_,
    v5 = date_
)
C.exe(stm)
C.print_table()

[{'id': 1, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 12, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'timestamp_without_tz': datetime.datetime(2025, 5, 26, 12, 32, 15, 468258), 'time_with_tz': datetime.time(12, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'time_without_tz': datetime.time(12, 32, 15, 468258), 'date_column': datetime.date(2025, 5, 26)}, {'id': 2, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 13, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'timestamp_without_tz': datetime.datetime(2025, 5, 26, 13, 4, 17, 280505), 'time_with_tz': datetime.time(13, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'time_without_tz': datetime.time(13, 4, 17, 280505), 'date_column': datetime.date(2025, 5, 26)}]


<h2>core and python</h2>

In [7]:
from datetime import datetime
from zoneinfo import ZoneInfo

timestamptz = datetime.now(ZoneInfo("Europe/Helsinki"))
timestamp = timestamptz.replace(tzinfo=None)

timetz = timestamptz.timetz()
time_ = timetz.replace(tzinfo=None)

date_ = timestamptz.date()

from sqlalchemy import insert
stm = insert(t).values(
    timestamp_with_tz=timestamptz,
    timestamp_without_tz=timestamp,
    time_with_tz=timetz,
    time_without_tz=time_,
    date_column=date_
)
C.exe(stm)
C.print_table()

[{'id': 1, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 12, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'timestamp_without_tz': datetime.datetime(2025, 5, 26, 12, 32, 15, 468258), 'time_with_tz': datetime.time(12, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'time_without_tz': datetime.time(12, 32, 15, 468258), 'date_column': datetime.date(2025, 5, 26)}, {'id': 2, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 13, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'timestamp_without_tz': datetime.datetime(2025, 5, 26, 13, 4, 17, 280505), 'time_with_tz': datetime.time(13, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'time_without_tz': datetime.time(13, 4, 17, 280505), 'date_column': datetime.date(2025, 5, 26)}, {'id': 3, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 13, 8, 4, 401263, tzinfo=datet

<h2>orm and python</h2>

In [15]:
from datetime import datetime
from zoneinfo import ZoneInfo

timestamptz = datetime.now(ZoneInfo("Europe/Helsinki"))
timestamp = timestamptz.replace(tzinfo=None)

timetz = timestamptz.timetz()
time_ = timetz.replace(tzinfo=None)

date_ = timestamptz.date()

row = ReflectedClass(
    content=None,
    timestamp_with_tz=timestamptz,
    timestamp_without_tz=timestamp,
    time_with_tz=timetz,
    time_without_tz=time_,
    date_column=date_
)

with Session(engine) as session:
    session.add(row)
    session.commit()
C.print_table()


[{'id': 1, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 12, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'timestamp_without_tz': datetime.datetime(2025, 5, 26, 12, 32, 15, 468258), 'time_with_tz': datetime.time(12, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'time_without_tz': datetime.time(12, 32, 15, 468258), 'date_column': datetime.date(2025, 5, 26)}, {'id': 2, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 13, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'timestamp_without_tz': datetime.datetime(2025, 5, 26, 13, 4, 17, 280505), 'time_with_tz': datetime.time(13, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'time_without_tz': datetime.time(13, 4, 17, 280505), 'date_column': datetime.date(2025, 5, 26)}, {'id': 3, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 13, 8, 4, 401263, tzinfo=datet

In [17]:
from datetime import datetime
from zoneinfo import ZoneInfo

timestamptz = datetime.now(ZoneInfo("Europe/Helsinki"))
timestamp = timestamptz.replace(tzinfo=None)

timetz = timestamptz.timetz()
time_ = timetz.replace(tzinfo=None)

date_ = timestamptz.date()

row = ReflectedClass(
    content=None,
    timestamp_with_tz=timestamptz,
    timestamp_without_tz=timestamp,
    time_with_tz=timetz,
    time_without_tz=time_,
    date_column=date_
)

with Session(engine) as session:
    session.add(row)
    session.commit()
ReflectedClass.print_table()


[{'id': 1, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 12, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'timestamp_without_tz': datetime.datetime(2025, 5, 26, 12, 32, 15, 468258), 'time_with_tz': datetime.time(12, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'time_without_tz': datetime.time(12, 32, 15, 468258), 'date_column': datetime.date(2025, 5, 26)}, {'id': 2, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 13, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'timestamp_without_tz': datetime.datetime(2025, 5, 26, 13, 4, 17, 280505), 'time_with_tz': datetime.time(13, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'time_without_tz': datetime.time(13, 4, 17, 280505), 'date_column': datetime.date(2025, 5, 26)}, {'id': 3, 'content': None, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 13, 8, 4, 401263, tzinfo=datet

<h1>starting tdate_orm</h1>

In [13]:
from sqlalchemy import create_engine
engine = create_engine(uri)

from sqlalchemy.orm import registry
sqlalchemy_registry = registry()

from sqlalchemy.ext.automap import automap_base
Base = automap_base(declarative_base= sqlalchemy_registry.generate_base())
Base.prepare(autoload_with= engine)

C = Base.classes.tdate_orm
t = C. __table__
timestamptz = t.columns.timestamp_with_tz

from sqlalchemy.orm import Session

@classmethod
def describe(cls):
    col_lst = cls.__table__.columns
    return {col.name: col.type for col in col_lst}

@classmethod
def print_table(cls):
    with Session(engine) as session:
        print(session.execute(select(cls.__table__)).mappings().all())

def repr_func(self):
    col_lst = self.__class__.__table__.columns
    row = [(col.name, getattr(self, col.name)) for col in col_lst]
    return f"Instance of class {self.__class__.__name__} (table {self.__class__.__tablename__}):\n {row}"

setattr(C, 'describe', describe)
setattr(C, 'print_table', print_table)
setattr(C, '__repr__', repr_func)

<h1>INTERVAL and timedelta</h1>

<h2>text</h2>

In [6]:
raw = """
SELECT timestamp_with_tz
- INTERVAL :i
FROM tdate_orm
"""
from sqlalchemy import text
stm = text(raw).bindparams(
    i = '2 YEAR 3 DAY 1 HOUR'
)
with Session(engine) as session:
    print(session.execute(stm).mappings().all())


[{'?column?': datetime.datetime(2023, 5, 23, 11, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'?column?': datetime.datetime(2023, 5, 23, 12, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'?column?': datetime.datetime(2023, 5, 23, 12, 8, 4, 401263, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'?column?': datetime.datetime(2023, 5, 23, 12, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'?column?': datetime.datetime(2023, 5, 23, 12, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'?column?': datetime.datetime(2023, 5, 23, 12, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}]


<h2>core and text</h2>

In [9]:
from sqlalchemy import select

stm = select(timestamptz - text("INTERVAL'2 YEAR 3 DAY 1 HOUR'"))

with Session(engine) as session:
    print(session.execute(stm).mappings().all())

[{'anon_1': datetime.datetime(2023, 5, 23, 11, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2023, 5, 23, 12, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2023, 5, 23, 12, 8, 4, 401263, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2023, 5, 23, 12, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2023, 5, 23, 12, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2023, 5, 23, 12, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}]


<h2>core and literal_column</h2>

In [10]:
from sqlalchemy import select, literal_column

stm = select(timestamptz - literal_column("INTERVAL'2 YEAR 3 DAY 1 HOUR'"))

with Session(engine) as session:
    print(session.execute(stm).mappings().all())

[{'anon_1': datetime.datetime(2023, 5, 23, 11, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2023, 5, 23, 12, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2023, 5, 23, 12, 8, 4, 401263, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2023, 5, 23, 12, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2023, 5, 23, 12, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2023, 5, 23, 12, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}]


<h2>core and cast(timedelta)</h2>

In [15]:
from sqlalchemy.dialects.postgresql import INTERVAL
from datetime import timedelta
from sqlalchemy import Cast, literal

td = timedelta(days = 3, hours= 1)
interval = Cast(literal(td), INTERVAL)

stm = select(timestamptz - interval)
with Session(engine) as session:
    print(session.execute(stm).mappings().all())


[{'anon_1': datetime.datetime(2025, 5, 23, 11, 32, 15, 468258, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2025, 5, 23, 12, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2025, 5, 23, 12, 8, 4, 401263, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2025, 5, 23, 12, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2025, 5, 23, 12, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'anon_1': datetime.datetime(2025, 5, 23, 12, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}]


<h2>pythonic end time calculation then cast</h2>

In [24]:
start_time_stm = select(timestamptz)
with Session(engine) as session:
    start_time_lst = session.scalars(start_time_stm).all()

td = timedelta(days = 3, hours= 1)
end_time_lst = [t - td for t in start_time_lst]

from sqlalchemy import TIMESTAMP
def stm_creator(t):
    return select(Cast(literal(t), TIMESTAMP(timezone= True)))
stm_list = [stm_creator(t) for t in end_time_lst]

with Session(engine) as session:
    for stm in stm_list:
        result = session.scalar(stm)
        print (result)

2025-05-23 11:32:15.468258+03:00
2025-05-23 12:04:17.280505+03:00
2025-05-23 12:08:04.401263+03:00
2025-05-23 12:23:18.402512+03:00
2025-05-23 12:36:20.086335+03:00
2025-05-23 12:40:34.475703+03:00


<h2>get start time data from db then full pythonic</h2>

In [27]:
start_time_stm = select(timestamptz)
with Session(engine) as session:
    start_time_lst = session.scalars(start_time_stm).all()

td = timedelta(days = 3, hours= 1)
end_time_lst = [t - td for t in start_time_lst]
for t in end_time_lst:
    print(t)

2025-05-23 11:32:15.468258+03:00
2025-05-23 12:04:17.280505+03:00
2025-05-23 12:08:04.401263+03:00
2025-05-23 12:23:18.402512+03:00
2025-05-23 12:36:20.086335+03:00
2025-05-23 12:40:34.475703+03:00


<h1>EXTRACT from timestamp</h1>

<h2>text</h2>

In [3]:
from sqlalchemy import text

In [5]:
raw = """
SELECT EXTRACT
(YEAR FROM timestamp_without_tz)
FROM tdate_orm
"""
with Session(engine) as session:
    print(session.execute(text(raw)).mappings().all())

[{'date_part': 2025.0}, {'date_part': 2025.0}, {'date_part': 2025.0}, {'date_part': 2025.0}, {'date_part': 2025.0}, {'date_part': 2025.0}]


<h2>core/orm</h2>

In [9]:
from sqlalchemy import func, select

In [10]:
extract_stm = func.extract('year', timestamptz)
stm = select(extract_stm)
with Session(engine) as session:
    print(session.execute(stm).mappings().all())

[{'anon_1': 2025.0}, {'anon_1': 2025.0}, {'anon_1': 2025.0}, {'anon_1': 2025.0}, {'anon_1': 2025.0}, {'anon_1': 2025.0}]


<h2>python</h2>

In [15]:
stm = select(timestamptz)
with Session(engine) as session:
    timestamptz_lst = session.scalars(stm).all()

year_lst = [t.year for t in timestamptz_lst]
year_lst

[2025, 2025, 2025, 2025, 2025, 2025]

<h1>AGE</h1>

<h2>text</h2>

In [20]:
raw = """
SELECT AGE
(NOW(), timestamp_with_tz)
AS new_interval
FROM tdate_orm
"""
with Session(engine) as session:
    print(session.execute(text(raw)).mappings().all())

[{'new_interval': datetime.timedelta(days=1, seconds=79988, microseconds=135946)}, {'new_interval': datetime.timedelta(days=1, seconds=78066, microseconds=323699)}, {'new_interval': datetime.timedelta(days=1, seconds=77839, microseconds=202941)}, {'new_interval': datetime.timedelta(days=1, seconds=76925, microseconds=201692)}, {'new_interval': datetime.timedelta(days=1, seconds=76143, microseconds=517869)}, {'new_interval': datetime.timedelta(days=1, seconds=75889, microseconds=128501)}]


<h2>core/orm</h2>

In [21]:
age_stm = func.age(func.now(), timestamptz).label('new_interval')
stm = select(age_stm)
with Session(engine) as session:
    print(session.execute(stm).mappings().all())

[{'new_interval': datetime.timedelta(days=1, seconds=80008, microseconds=950059)}, {'new_interval': datetime.timedelta(days=1, seconds=78087, microseconds=137812)}, {'new_interval': datetime.timedelta(days=1, seconds=77860, microseconds=17054)}, {'new_interval': datetime.timedelta(days=1, seconds=76946, microseconds=15805)}, {'new_interval': datetime.timedelta(days=1, seconds=76164, microseconds=331982)}, {'new_interval': datetime.timedelta(days=1, seconds=75909, microseconds=942614)}]


<h2>core/orm and python</h2>

In [23]:
from datetime import datetime
from sqlalchemy import literal
from zoneinfo import ZoneInfo

In [24]:
age_stm = func.age(literal(datetime.now(ZoneInfo("Europe/Helsinki"))), timestamptz).label('new_interval')
stm = select(age_stm)
with Session(engine) as session:
    print(session.execute(stm).mappings().all())

[{'new_interval': datetime.timedelta(days=1, seconds=80123, microseconds=627589)}, {'new_interval': datetime.timedelta(days=1, seconds=78201, microseconds=815342)}, {'new_interval': datetime.timedelta(days=1, seconds=77974, microseconds=694584)}, {'new_interval': datetime.timedelta(days=1, seconds=77060, microseconds=693335)}, {'new_interval': datetime.timedelta(days=1, seconds=76279, microseconds=9512)}, {'new_interval': datetime.timedelta(days=1, seconds=76024, microseconds=620144)}]


<h2>python</h2>

In [25]:
stm = select(timestamptz)
with Session(engine) as session:
    timestamptz_lst = session.scalars(stm).all()
now_ = datetime.now(ZoneInfo("Europe/Helsinki"))
interval_lst = [now_ - t for t in timestamptz_lst]
interval_lst

[datetime.timedelta(days=1, seconds=80222, microseconds=104137),
 datetime.timedelta(days=1, seconds=78300, microseconds=291890),
 datetime.timedelta(days=1, seconds=78073, microseconds=171132),
 datetime.timedelta(days=1, seconds=77159, microseconds=169883),
 datetime.timedelta(days=1, seconds=76377, microseconds=486060),
 datetime.timedelta(days=1, seconds=76123, microseconds=96692)]

<h1>extracting second from interval</h1>

<h2>text</h2>

In [26]:
raw = """
SELECT EXTRACT
(SECOND FROM AGE
                (NOW(), timestamp_with_tz))
FROM tdate_orm
"""
with Session(engine) as session:
    print(session.execute(text(raw)).mappings().all())

[{'date_part': 11.115939}, {'date_part': 9.303692}, {'date_part': 22.182934}, {'date_part': 8.181685}, {'date_part': 6.497862}, {'date_part': 52.108494}]


<h2>core</h2>

In [27]:
interval = func.age(func.now(), timestamptz)
sec = func.extract('second', interval)
stm = select(sec)
with Session(engine) as session:
    print(session.execute(stm).mappings().all())

[{'anon_1': 28.690036}, {'anon_1': 26.877789}, {'anon_1': 39.757031}, {'anon_1': 25.755782}, {'anon_1': 24.071959}, {'anon_1': 9.682591}]


<h2>python</h2>

In [39]:
stm = select(timestamptz)
with Session(engine) as session:
    timestamptz_lst = session.scalars(stm).all()
now_ = datetime.now(ZoneInfo("Europe/Helsinki"))

def get_sec(td):
    sec1 = td.seconds % 60
    sec2 = td.microseconds / (10**6)
    return sec1 + sec2
sec_lst = [get_sec(now_ - t) for t in timestamptz_lst]
sec_lst

[36.643864, 34.831617, 47.710859, 33.70961, 32.025787, 17.636419]

<h1>LIMIT and OFFSET</h1>

<h2>text</h2>

In [5]:
from sqlalchemy import text, select
from sqlalchemy.orm import Session

raw = """
SELECT id, timestamp_with_tz
FROM tdate_orm
LIMIT 2
OFFSET 1
"""

with Session(engine) as session:
    print(session.execute(text(raw)).mappings().all())

[{'id': 2, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 13, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'id': 3, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 13, 8, 4, 401263, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}]


<h2>core/orm</h2>

In [6]:
stm = select(t.c.id, timestamptz).limit(2).offset(1)
with Session(engine) as session:
    print(session.execute(stm).mappings().all())

[{'id': 2, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 13, 4, 17, 280505, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}, {'id': 3, 'timestamp_with_tz': datetime.datetime(2025, 5, 26, 13, 8, 4, 401263, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800)))}]


# constraints in postgres

<table>
  <thead>
    <tr>
      <th>Constraint Type</th>
      <th>Keyword</th>
      <th>Short Definition</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><strong>Primary Key</strong></td>
      <td><code>PRIMARY KEY</code></td>
      <td>Ensures each row has a <strong>unique, non-null</strong> identifier. Only one per table.</td>
    </tr>
    <tr>
      <td><strong>Unique</strong></td>
      <td><code>UNIQUE</code></td>
      <td>Guarantees that a column (or group of columns) has <strong>unique values</strong> across rows. Allows nulls.</td>
    </tr>
    <tr>
      <td><strong>Not Null</strong></td>
      <td><code>NOT NULL</code></td>
      <td>Ensures that a column <strong>cannot contain null values</strong>.</td>
    </tr>
    <tr>
      <td><strong>Check</strong></td>
      <td><code>CHECK</code></td>
      <td>Enforces a <strong>boolean expression</strong> on values (e.g., <code>age &gt; 0</code>).</td>
    </tr>
    <tr>
      <td><strong>Foreign Key</strong></td>
      <td><code>FOREIGN KEY</code></td>
      <td>Enforces <strong>referential integrity</strong> by requiring values to match values in another table’s primary key.</td>
    </tr>
    <tr>
      <td><strong>Exclusion</strong></td>
      <td><code>EXCLUDE</code></td>
      <td>Prevents rows from <strong>conflicting</strong> with each other on a specified condition (e.g., overlapping time ranges). PostgreSQL-specific.</td>
    </tr>
    <tr>
      <td><strong>Default</strong></td>
      <td><code>DEFAULT</code></td>
      <td>Assigns a <strong>default value</strong> to a column if no value is provided on insert. <em>(Not technically a constraint, but often grouped here.)</em></td>
    </tr>
    <tr>
      <td><strong>Deferrable</strong></td>
      <td><code>DEFERRABLE</code></td>
      <td>Controls <strong>when</strong> constraint checks occur (immediately or at the end of a transaction). Applies to PK, FK, and UNIQUE.</td>
    </tr>
    <tr>
      <td><strong>Expression Index</strong></td>
      <td><em>(via indexes)</em></td>
      <td>Not a "constraint" per se, but sometimes used in place of <code>CHECK</code> for performance with expressions (e.g., <code>WHERE deleted = false</code>).</td>
    </tr>
    <tr>
      <td><strong>Generated Column</strong></td>
      <td><code>GENERATED ALWAYS AS</code></td>
      <td>Derives a column’s value from an expression. Read-only. <em>(PostgreSQL 12+)</em></td>
    </tr>
  </tbody>
</table>


# general syntax for adding and dropping primary key, check and unique constraints

<h1>finding the primary key of a table</h1>

In [7]:
from sqlalchemy import inspect

inspector = inspect(engine)

In [8]:
inspector.get_pk_constraint('tdate_orm')

{'constrained_columns': ['id'], 'name': 'tdate_orm_pkey', 'comment': None}

<h1>remove pk constraint</h1>

In [9]:
inspector.get_pk_constraint('core_table')

{'constrained_columns': ['id'], 'name': 'core_table_pkey', 'comment': None}

In [19]:
inspector.get_pk_constraint('core_table')['name']

'core_table_pkey'

In [ ]:
raw = """
ALTER TABLE core_table
DROP CONSTRAINT
core_table_pkey
"""

with Session(engine) as session:
    session.execute(text(raw))
    session.commit()

In [15]:
inspector = inspect(engine)
inspector.get_pk_constraint('core_table')

{'name': None, 'constrained_columns': []}

<h1>add pk constraint</h1>

In [16]:
raw = """
ALTER TABLE core_table
ADD
PRIMARY KEY (id)
"""

with Session(engine) as session:
    session.execute(text(raw))
    session.commit()

In [17]:
inspector = inspect(engine)
inspector.get_pk_constraint('core_table')

{'constrained_columns': ['id'], 'name': 'core_table_pkey', 'comment': None}

<h1>deleting rows</h1>

## text

In [20]:
raw = """
DELETE
FROM tdate_orm
WHERE id = 1
"""

with Session(engine) as session:
    session.execute(text(raw))
    session.commit()

## core

In [21]:
from sqlalchemy import delete

stm = delete(t).where(t.c.id == 2)
with Session(engine) as session:
    session.execute(stm)
    session.commit()

## orm

In [22]:
stm = delete(C).where(C.id == 3)
with Session(engine) as session:
    session.execute(stm)
    session.commit()

In [23]:
with Session(engine) as seesion:
    print(session.execute(select(t)).all())

[(4, None, datetime.datetime(2025, 5, 26, 13, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 23, 18, 402512), datetime.time(13, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 23, 18, 402512), datetime.date(2025, 5, 26)), (5, None, datetime.datetime(2025, 5, 26, 13, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 36, 20, 86335), datetime.time(13, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 36, 20, 86335), datetime.date(2025, 5, 26)), (6, None, datetime.datetime(2025, 5, 26, 13, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 40, 34, 475703), datetime.time(13, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 40, 34, 475703), datetime.date(2025, 5, 26))

# Update rows

## text

In [ ]:
raw = """
UPDATE tdate_orm
SET content = 'nima'
WHERE id = 4
"""

with Session(engine) as session:
    session.execute(text(raw))
    session.commit()

In [25]:
with Session(engine) as seesion:
    print(session.execute(select(t).order_by(t.c.id)).all())

[(4, 'nima', datetime.datetime(2025, 5, 26, 13, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 23, 18, 402512), datetime.time(13, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 23, 18, 402512), datetime.date(2025, 5, 26)), (5, None, datetime.datetime(2025, 5, 26, 13, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 36, 20, 86335), datetime.time(13, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 36, 20, 86335), datetime.date(2025, 5, 26)), (6, None, datetime.datetime(2025, 5, 26, 13, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 40, 34, 475703), datetime.time(13, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 40, 34, 475703), datetime.date(2025, 5, 26

## core / orm

In [26]:
from sqlalchemy import update

stm = update(t).where(t.c.id == 5).values(content = 'banafsheh')
with Session(engine) as session:
    session.execute(stm)
    session.commit()

In [27]:
with Session(engine) as seesion:
    print(session.execute(select(t).order_by(t.c.id)).all())

[(4, 'nima', datetime.datetime(2025, 5, 26, 13, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 23, 18, 402512), datetime.time(13, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 23, 18, 402512), datetime.date(2025, 5, 26)), (5, 'banafsheh', datetime.datetime(2025, 5, 26, 13, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 36, 20, 86335), datetime.time(13, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 36, 20, 86335), datetime.date(2025, 5, 26)), (6, None, datetime.datetime(2025, 5, 26, 13, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 40, 34, 475703), datetime.time(13, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 40, 34, 475703), datetime.date(2025

# UNIQUE vs PRIMARY KEY

# Defining a uniqe constraint

# finding UNIQUE CONSTRAINT

In [3]:
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_unique_constraints('tdate_orm')

[]

# adding UNIQUE

In [4]:
raw = """
ALTER TABLE tdate_orm
ADD
UNIQUE (content)
"""

from sqlalchemy import text
with Session(engine) as session:
    session.execute(text(raw))
    session.commit()

In [5]:
inspector = inspect(engine)
inspector.get_unique_constraints('tdate_orm')

[{'column_names': ['content'],
  'name': 'tdate_orm_content_key',
  'comment': None}]

# droping UNIQUE

In [7]:
inspector.get_unique_constraints('tdate_orm')[0]['name']

'tdate_orm_content_key'

In [8]:
raw = """
ALTER TABLE tdate_orm
DROP
CONSTRAINT tdate_orm_content_key
"""

with Session(engine) as session:
    session.execute(text(raw))
    session.commit()

In [ ]:
inspector = inspect(engine)
inspector.get_unique_constraints('tdate_orm')

# check constraint

# finding check constraint

In [7]:
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_check_constraints('tdate_orm')

[]

# add check constraint

In [8]:
from sqlalchemy import text

In [11]:
raw = """
ALTER TABLE tdate_orm
ADD CONSTRAINT
constraint_name
CHECK (content = 'nima' OR content = 'banafsheh')
"""

with Session(engine) as session:
    session.execute(text(raw))
    session.commit()
inspector = inspect(engine)
inspector.get_check_constraints('tdate_orm')

[{'name': 'constraint_name',
  'sqltext': "content::text = 'nima'::text OR content::text = 'banafsheh'::text",
  'comment': None}]

# Drop check constraint

In [12]:
raw = """
ALTER TABLE tdate_orm
DROP CONSTRAINT
constraint_name
"""

with Session(engine) as session:
    session.execute(text(raw))
    session.commit()
inspector = inspect(engine)
inspector.get_check_constraints('tdate_orm')

[]

# CONSTRAINT in ORM model and Table()

## ORM model

In [6]:
from sqlalchemy.orm import registry
from sqlalchemy import Table
sqlalchemy_registry = registry()
mapped = sqlalchemy_registry.mapped
metadata = sqlalchemy_registry.metadata

from sqlalchemy import Column, Integer, String, CheckConstraint

@mapped
class C:
    __tablename__= 'constraint_orm'
    __table_args__= (
        CheckConstraint("gender IN ('male', 'female')", name = "check_gender"),
    )
    id = Column(Integer, primary_key= True, autoincrement= True)
    email = Column(String, unique= True)
    gender = Column(String)

metadata.create_all(engine)

## Core

In [7]:
Table(
    'constraint_core',
    metadata,
    Column('id', Integer, primary_key= True, autoincrement= True),
    Column('email', String, unique= True, nullable= False),
    Column('gender', String),
    CheckConstraint("gender IN ('male', 'female')", name = "check_gender")
)
metadata.create_all(engine)

# ENUM

# create ENUM

## text

In [3]:
raw = """
CREATE TYPE type_name
AS
ENUM ('value1', 'value2', 'value3')
"""

from sqlalchemy import text
from sqlalchemy.orm import Session
with Session(engine) as session:
    session.execute(text(raw))
    session.commit()

## python and core/orm

In [ ]:
#step 1
import enum

class PythonEnum(enum.Enum):
    v_1 = 'value1'
    v_2 = 'value2'
    v_3 = 'value3'

#step 2
from sqlalchemy import Enum
new_type = Enum(PythonEnum, name = 'type_name_2')

# using already defined Enum

## text

In [4]:
raw = """
CREATE TABLE enum_table
(
    id SERIAL PRIMARY KEY,
    enum_col type_name
)
"""

with Session(engine) as session:
    session.execute(text(raw))
    session.commit()

## core/orm

In [18]:
from sqlalchemy import Enum
import enum

# dobare baiad class enum python ro benevisi
class PythonEnum(enum.Enum):
    v_1 = 'value1'
    v_2 = 'value2'
    v_3 = 'value3'
# new_type daghighan esme enum dar db hast
already_deifned_enum = Enum(
    PythonEnum,
    name = 'type_name',
    native_enum = True,
    create_type = False
)

from sqlalchemy.orm import registry

sqlalchemy_registry = registry()
mapped = sqlalchemy_registry.mapped
metadata = sqlalchemy_registry.metadata

from sqlalchemy import Integer, Column

@mapped
class C2:
    __tablename__ = 'orm_enum_2'
    id = Column(Integer, primary_key= True, autoincrement= True)
    enum_col = Column(already_deifned_enum)

metadata.create_all(engine)

# DDL

In [3]:
raw = """
CREATE TABLE tddl
(
    id SERIAL PRIMARY KEY,
    content VARCHAR
)
"""

from sqlalchemy import DDL
ddl_stm = DDL(raw)

from sqlalchemy.orm import Session
with Session(engine) as session:
    session.execute(ddl_stm)
    session.commit()
    

# event

| **Event Layer**       | **Useful For**                        | **Example**                                        |
|-----------------------|----------------------------------------|----------------------------------------------------|
| `DDL Events`          | Schema setup and teardown              | Add index, extension, trigger after table creation |
| `Mapper Events`       | ORM object lifecycle (insert, update)  | Modify data before insert                          |
| `Session Events`      | Session-level commits/flushes          | Audit logs, enforce policies                       |
| `Connection Events`   | DB connection-specific hooks           | Logging queries or enforcing startup checks        |


In [4]:
from sqlalchemy.orm import registry
sqlalchemy_registry = registry()
metadata = sqlalchemy_registry.metadata

from sqlalchemy import Table, Column, Integer, String
table_obj = Table ('first',
                   metadata,
                   Column('id', Integer, autoincrement=True  ,primary_key= True),
                   Column('content', String)
                  )
    


raw = """
CREATE TABLE second
(
    id SERIAL PRIMARY KEY,
    content VARCHAR
)
"""

ddl_obj = DDL(raw)

from sqlalchemy import event
event.listen(table_obj , 'after_create', ddl_obj)

metadata.create_all(engine)

In [5]:
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_table_names()

['constraint_core',
 'core_table',
 'new_table',
 'new_table_2',
 'enum_table',
 'orm_enum',
 'orm_enum_2',
 'declaritive_table',
 'registery_table',
 'core_orm',
 'tddl',
 'first',
 'second',
 'tdate_orm',
 'tdate_text',
 'constraint_orm']

# sqlalchemy event vs python asyncio

# transaction, flush and commit

## Transaction

A **transaction** is one or more operations done together to the database as a single atomic unit.  
If any operation in the transaction fails, all operations are undone (rolled back) to the state before the start of the transaction.  

The key is that transactions allow for intermediate states or “halfway transactions” during their execution, but these are **not permanent until commit**.

### Additional notes:
- Think of the transaction as a container holding all changes until you decide to finalize (`commit`) or discard (`rollback`) them.  
- These intermediate states (after `flush`, for example) are **only visible inside the ongoing transaction**.

---

## Flush

**Flush** is an operation that can occur during a transaction to send pending changes from the session to the database.  
This means the effects of prior operations become visible to subsequent operations within the same transaction.  
However, the changes from a flush are **not permanent**; if the transaction eventually rolls back, all flush effects are undone as well.

### Additional notes:
- Flush is useful when you want to ensure the database reflects your current session state temporarily (e.g., for integrity checks or queries dependent on new data).  
- Flush happens automatically in SQLAlchemy before queries or commits, but you can also trigger it manually. Reads (SELECT queries) and commits cause automatic flushes, but adds, updates, and deletes alone do not—those wait until flush or commit.

---

## Commit

**Commit** marks the end of a transaction, permanently saving all changes made during that transaction to the database.  
Any operations after commit belong to a new transaction.  
If a transaction ends without a commit (for example, if it’s closed or an error occurs), all changes including any flushes are rolled back to the state before the transaction began.

### Additional notes:
- Commit is the only operation that truly makes changes durable and visible outside the current transaction scope.  
- Without commit, all changes are ephemeral and discarded on rollback.

---

## How SQLAlchemy’s Session context manager fits in

In SQLAlchemy, using a **Session** as a context manager (e.g., `with Session() as session:`) provides a clean, structured way to start and wrap up a transaction.  

At the end of the `with` block:
- If you explicitly call `session.commit()`, the transaction is committed, making your changes permanent.  
- If you don’t commit, or if an error occurs, the session will **rollback automatically** when the context manager closes, reverting the database to the state before the session began.  
- The context manager also takes care of cleaning up resources, closing connections, and preventing leaks.


# running .sql file

In [12]:
from sqlalchemy import text
from sqlalchemy.orm import Session

In [8]:
from sqlalchemy import inspect
inspect(engine).get_table_names()

['constraint_core',
 'core_table',
 'new_table',
 'new_table_2',
 'enum_table',
 'orm_enum',
 'orm_enum_2',
 'declaritive_table',
 'registery_table',
 'core_orm',
 'tddl',
 'first',
 'second',
 'tdate_orm',
 'tdate_text',
 'constraint_orm']

In [10]:
with open('stm.sql') as file:
    statements = file.read()
with Session(engine) as session:
    session.execute(text(statements))
    session.commit()

In [11]:
inspect(engine).get_table_names()

['users_file',
 'constraint_core',
 'core_table',
 'new_table',
 'new_table_2',
 'enum_table',
 'orm_enum',
 'orm_enum_2',
 'declaritive_table',
 'registery_table',
 'core_orm',
 'tddl',
 'first',
 'second',
 'tdate_orm',
 'tdate_text',
 'constraint_orm']

# ON CONFLICT DO

In [14]:
from sqlalchemy import select
with Session(engine) as session:
    print(session.execute(select(t)).all())

[(6, None, datetime.datetime(2025, 5, 26, 13, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 40, 34, 475703), datetime.time(13, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 40, 34, 475703), datetime.date(2025, 5, 26)), (4, 'nima', datetime.datetime(2025, 5, 26, 13, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 23, 18, 402512), datetime.time(13, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 23, 18, 402512), datetime.date(2025, 5, 26)), (5, 'banafsheh', datetime.datetime(2025, 5, 26, 13, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 36, 20, 86335), datetime.time(13, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 36, 20, 86335), datetime.date(2025

## text

In [ ]:
with Session(engine) as session:
    session.execute(text("""INSERT INTO tdate_orm
                            (id)
                            VALUES
                            (6)
                            """))
    session.commit()

# IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "tdate_orm_pkey"
# DETAIL:  Key (id)=(6) already exists.

In [17]:
with Session(engine) as session:
    session.execute(text("""INSERT INTO tdate_orm
                            (id)
                            VALUES
                            (6)
                            ON CONFLICT (id)
                            DO NOTHING
                            """))
    session.commit()

## core

In [24]:
from sqlalchemy.dialects.postgresql import insert

In [25]:
stm = insert(t).values(id = 6, content = 'asal')

In [26]:
stm = stm.on_conflict_do_update(
    index_elements=['id'],
    set_={
        'content': stm.excluded.content
    }
)

In [27]:
with Session(engine) as session:
    session.execute(stm)
    session.commit()

In [29]:
with Session(engine) as session:
    print(session.execute(select(t)).all())

[(4, 'nima', datetime.datetime(2025, 5, 26, 13, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 23, 18, 402512), datetime.time(13, 23, 18, 402512, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 23, 18, 402512), datetime.date(2025, 5, 26)), (5, 'banafsheh', datetime.datetime(2025, 5, 26, 13, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 36, 20, 86335), datetime.time(13, 36, 20, 86335, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 36, 20, 86335), datetime.date(2025, 5, 26)), (6, 'asal', datetime.datetime(2025, 5, 26, 13, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.datetime(2025, 5, 26, 13, 40, 34, 475703), datetime.time(13, 40, 34, 475703, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), datetime.time(13, 40, 34, 475703), datetime.date(20

## orm

# Foreign Key

# Defining a one to many Foregn Key

## Inline Foreign Key Definition (Column-level)

### text

### core/orm

## Table-level Foreign Key Definition (Named Constraint)

### text

### core

## Add Foreign Key Constraint Using ALTER TABLE

### text

# Defining a one to one Foregn Key

### Inline Foreign Key Definition (Column-level)

### text

### core/orm

## other ways

# Relation

# example of two table with relation

In [23]:
from sqlalchemy.orm import registry, Session
from sqlalchemy import Column, Integer, String , ForeignKey

sqlalchemy_registry = registry()
mapped = sqlalchemy_registry.mapped
metadata = sqlalchemy_registry.metadata

@mapped
class C1:
    __tablename__='pet'
    id = Column (Integer, autoincrement= True, primary_key= True)
    pet_name = Column(String)

@mapped
class C2:
    __tablename__='owner'
    id = Column (Integer, autoincrement= True, primary_key= True)
    owner_name = Column(String)
    pet_id = Column(Integer, ForeignKey('pet.id'))

metadata.create_all(engine)

# relatioship vs Foreign Key

# syntax of relationship

# use of relationship

# example of relationship

In [3]:
from sqlalchemy.orm import registry, relationship
from sqlalchemy import Column, Integer, String , ForeignKey

sqlalchemy_registry = registry()
mapped = sqlalchemy_registry.mapped

@mapped
class Pet:
    __tablename__='pet'
    id = Column (Integer, autoincrement= True, primary_key= True)
    pet_name = Column(String)

    def __repr__(self):
        return f'a pet named {self.pet_name}'

    # relationship attr
    owners = relationship ('Owner', back_populates='pet')

@mapped
class Owner:
    __tablename__='owner'
    id = Column (Integer, autoincrement= True, primary_key= True)
    owner_name = Column(String)
    pet_id = Column(Integer, ForeignKey('pet.id'))

    def __repr__(self):
        return f'an owner named {self.owner_name}'

    # relationship attr
    pet = relationship ('Pet', back_populates='owners')

# using relatioship instead of select

## lazy loading

In [22]:
from sqlalchemy.orm import Session
from sqlalchemy import select

with Session(engine) as session:
    asal = session.scalar(select(Pet).where(Pet.id == 1))
    owner_lst = asal.owners # lazy loading
    # iany baiad hanuz tu session bashim chon
    # asal ke az select be dast umad owners hash bedast nemiad be surat automatic,
    # mizre vaghti be khastim beheshun dast peida konim,
    # ie select dige ersal mikone

In [20]:
asal

a pet named asal

In [21]:
owner_lst

[an owner named nima, an owner named banafsheh]

## eager loading

### joined load

In [24]:
from sqlalchemy.orm import joinedload

stmt = select(Pet).options(joinedload(Pet.owners)).where(Pet.id == 1)
with Session(engine) as session:
    joinedload_asal = session.scalar(stmt)
joinedload_asal.owners

[an owner named nima, an owner named banafsheh]

### selection load

In [25]:
from sqlalchemy.orm import selectinload

stmt = select(Pet).options(selectinload(Pet.owners)).where(Pet.id == 1)

with Session(engine) as session:
    selectinload_asal = session.scalar(stmt)
selectinload_asal.owners

[an owner named nima, an owner named banafsheh]

# using relationship instead of update

In [6]:
from sqlalchemy.orm import Session

new_pet = Pet(pet_name = 'bambi')
with Session(engine) as session:
    session.add(new_pet)
    session.commit()

In [9]:
from sqlalchemy import select

with Session(engine) as session:
    banafseh = session.scalar(select(Owner).where(Owner.id == 2))

In [17]:
with Session(engine) as session:
    banafseh = session.scalar(select(Owner).where(Owner.id == 2))
    bambi = session.scalar(select(Pet).where(Pet.id == 2))
    banafseh.pet = bambi
    session.commit()

In [18]:
from sqlalchemy.orm import selectinload

with Session(engine) as session:
    banafsheh = session.scalar(select(Owner).options(selectinload(Owner.pet)).where(Owner.id == 2))

In [19]:
banafsheh.pet

a pet named bambi

In [20]:
banafsheh.pet_id

2

In [27]:
# moghaiese ba update mamuli

In [23]:
# with Session(engine) as session:
#     banafseh = session.scalar(select(Owner).where(Owner.id == 2))
#     bambi = session.scalar(select(Pet).where(Pet.id == 2))
#     banafseh.pet = bambi
#     session.commit()

with Session(engine) as session:
    session.execute(update(Owner).where(Owner.id == 2).values(pet_id = 1))
    session.commit()

In [26]:
with Session(engine) as session:
    banafsheh = session.scalar(select(Owner).options(selectinload(Owner.pet)).where(Owner.id == 2))
banafsheh.pet, banafsheh.pet_id

(a pet named asal, 1)

# Join

# Inner Join

In [26]:
from sqlalchemy.orm import registry
sqlalchemy_registry = registry()
from sqlalchemy.ext.automap import automap_base
Base = automap_base(sqlalchemy_registry.generate_base())
Base.prepare(autoload_with=engine)

In [27]:
Pet = Base.classes.pet
Owner = Base.classes.owner

In [28]:
asal = Pet(pet_name = 'asal')
nima = Owner(owner_name = 'nima', pet_id = 1)
banafsheh = Owner(owner_name = 'banafsheh', pet_id = 1)

from sqlalchemy.orm import Session
with Session(engine) as session:
    session.add(asal)
    session.flush()
    session.add(nima)
    session.add(banafsheh)
    session.commit()

## text

### *

In [31]:
from sqlalchemy import text
with Session(engine) as session:
    print(session.execute(text("""
    SELECT * FROM
    owner JOIN pet
    on owner.pet_id = pet.id
    """
                              )).mappings().all())

[{'id': 1, 'owner_name': 'nima', 'pet_id': 1, 'pet_name': 'asal'}, {'id': 1, 'owner_name': 'banafsheh', 'pet_id': 1, 'pet_name': 'asal'}]


### some columns

In [32]:
with Session(engine) as session:
    print(session.execute(text("""
    SELECT owner.owner_name, pet.pet_name FROM
    owner JOIN pet
    on owner.pet_id = pet.id
    """
                              )).mappings().all())

[{'owner_name': 'nima', 'pet_name': 'asal'}, {'owner_name': 'banafsheh', 'pet_name': 'asal'}]


## core

### *

In [43]:
from sqlalchemy import select
stm = select(Owner.__table__, Pet.__table__).join(Pet.__table__, Owner.pet_id == Pet.id)
with Session(engine) as session:
    print(session.execute(stm).mappings().all())

[{'id': 1, 'owner_name': 'nima', 'pet_id': 1, 'id_1': 1, 'pet_name': 'asal'}, {'id': 2, 'owner_name': 'banafsheh', 'pet_id': 1, 'id_1': 1, 'pet_name': 'asal'}]


### some columns

In [46]:
stm = select(Owner.__table__.c.owner_name, Pet.__table__.c.pet_name).join(Pet.__table__, Owner.pet_id == Pet.id)
with Session(engine) as session:
    print(session.execute(stm).mappings().all())

[{'owner_name': 'nima', 'pet_name': 'asal'}, {'owner_name': 'banafsheh', 'pet_name': 'asal'}]


## orm

### *

In [44]:
Owner.__repr__ = lambda self: f'owner:{self.owner_name}'
Pet.__repr__ = lambda self: f'pet:{self.pet_name}'

from sqlalchemy import select
stm = select(Owner, Pet).join(Pet, Owner.pet_id == Pet.id)
with Session(engine) as session:
    print(session.execute(stm).all())

[(owner:nima, pet:asal), (owner:banafsheh, pet:asal)]


### some columns

In [47]:
from sqlalchemy import select
stm = select(Owner.owner_name, Pet.pet_name).join(Pet, Owner.pet_id == Pet.id)
with Session(engine) as session:
    print(session.execute(stm).all())

[('nima', 'asal'), ('banafsheh', 'asal')]


# Automap vs Write the schema manually

# many to many relationship

In [ ]:
from sqlalchemy import Table, Column, Integer, String, PrimaryKeyConstraint, ForeignKey, select
from sqlalchemy.orm import registry, relationship, Session

sqlalchemy_registry = registry()
mapped = sqlalchemy_registry.mapped
metadata = sqlalchemy_registry.metadata

ownership = Table(
    'ownership',
    metadata,
    Column('owner_id', Integer, ForeignKey('new_owner.id', ondelete= 'cascade')),
    Column('pet_id', Integer, ForeignKey('new_pet.id', ondelete= 'cascade')),
    PrimaryKeyConstraint('owner_id', 'pet_id')
)

@mapped
class Owner:
    __tablename__ = 'new_owner'
    id = Column(Integer, autoincrement=True, primary_key= True)
    owner_name = Column (String, nullable= False)
    pets = relationship('Pet', back_populates='owners', secondary='ownership', passive_deletes= True)

@mapped
class Pet:
    __tablename__ = 'new_pet'
    id = Column(Integer, autoincrement=True, primary_key= True)
    pet_name = Column (String, nullable= False)
    owners = relationship('Owner', back_populates= 'pets', secondary= 'ownership', passive_deletes= True)
    

In [15]:
metadata.create_all(engine)

# using append and extend for many to many 

In [16]:
with Session(engine) as session:
    asal = Pet(pet_name ='asal')
    bambi = Pet(pet_name ='bambi')
    nima = Owner(owner_name = 'nima')
    banafsheh = Owner(owner_name = 'banafsheh')
    session.add_all([asal, bambi, nima, banafsheh])
    session.flush()
    nima.pets.append(asal)
    nima.pets.append(bambi)
    banafsheh.pets.extend([asal, bambi])
    session.commit()

In [18]:
with Session(engine) as session:
    print(session.execute(select(ownership)).mappings().all())

[{'owner_id': 1, 'pet_id': 1}, {'owner_id': 1, 'pet_id': 2}, {'owner_id': 2, 'pet_id': 1}, {'owner_id': 2, 'pet_id': 2}]


# using remove for many to many

In [19]:
with Session(engine) as session:
    bambi = session.scalar(select(Pet).where(Pet.id == 2))
    banafsheh = session.scalar(select(Owner).where(Owner.id == 2))
    banafsheh.pets.remove(bambi)
    session.commit()
                            

In [20]:
with Session(engine) as session:
    print(session.execute(select(ownership)).mappings().all())

[{'owner_id': 1, 'pet_id': 1}, {'owner_id': 1, 'pet_id': 2}, {'owner_id': 2, 'pet_id': 1}]


# text and many to many

In [3]:
raw = """
CREATE TABLE owner3 (
    id SERIAL PRIMARY KEY,
    owner_name VARCHAR NOT NULL
);

CREATE TABLE pet3 (
    id SERIAL PRIMARY KEY,
    pet_name VARCHAR NOT NULL
);

CREATE TABLE ownership3 (
    owner_id INTEGER REFERENCES owner3(id) ON DELETE CASCADE,
    pet_id INTEGER REFERENCES pet3(id) ON DELETE CASCADE,
    PRIMARY KEY(owner_id, pet_id)
);
"""
from sqlalchemy import text
from sqlalchemy.orm import Session
with Session(engine) as session:
    session.execute(text(raw))
    session.commit()

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [4]:
from sqlalchemy import inspect
inspect(engine).get_table_names()

['users_file',
 'constraint_core',
 'pet',
 'owner',
 'new_owner',
 'ownership',
 'new_pet',
 'owner3',
 'ownership3',
 'pet3',
 'core_table',
 'new_table',
 'new_table_2',
 'enum_table',
 'orm_enum',
 'orm_enum_2',
 'declaritive_table',
 'registery_table',
 'core_orm',
 'tddl',
 'first',
 'second',
 'tdate_orm',
 'tdate_text',
 'constraint_orm']

# inset into many to many via text

In [5]:
raw = """
INSERT INTO pet3
(pet_name)
VALUES
('asal'),
('bambi') ;

INSERT INTO owner3
(owner_name)
VALUES
('nima'),
('banafsheh') ;

INSERT INTO ownership3
(owner_id, pet_id)
VALUES
(1, 1),
(1, 2),
(2, 1) ;
"""

with Session(engine) as session:
    session.execute(text(raw))
    session.commit()

# inner join on many to many

## text

In [6]:
raw = """
SELECT owner3.owner_name, pet3.pet_name
FROM owner3
JOIN ownership3 ON ownership3.owner_id = owner3.id
JOIN pet3 ON ownership3.pet_id = pet3.id
"""
with Session(engine) as session:
    print(session.execute(text(raw)).mappings().all())

[{'owner_name': 'nima', 'pet_name': 'asal'}, {'owner_name': 'nima', 'pet_name': 'bambi'}, {'owner_name': 'banafsheh', 'pet_name': 'asal'}]


## core/orm

In [5]:
from sqlalchemy import Table, Column, Integer, String, PrimaryKeyConstraint, ForeignKey, select
from sqlalchemy.orm import registry, relationship, Session

sqlalchemy_registry = registry()
mapped = sqlalchemy_registry.mapped
metadata = sqlalchemy_registry.metadata

ownership = Table(
    'ownership3',
    metadata,
    Column('owner_id', Integer, ForeignKey('owner3.id', ondelete= 'cascade')),
    Column('pet_id', Integer, ForeignKey('pet3.id', ondelete= 'cascade')),
    PrimaryKeyConstraint('owner_id', 'pet_id')
)

@mapped
class Owner:
    __tablename__ = 'owner3'
    id = Column(Integer, autoincrement=True, primary_key= True)
    owner_name = Column (String, nullable= False)
    pets = relationship('Pet', back_populates='owners', secondary='ownership3', passive_deletes= True)

@mapped
class Pet:
    __tablename__ = 'pet3'
    id = Column(Integer, autoincrement=True, primary_key= True)
    pet_name = Column (String, nullable= False)
    owners = relationship('Owner', back_populates= 'pets', secondary= 'ownership3', passive_deletes= True)
    

In [7]:
stmt = select(Owner.owner_name, Pet.pet_name).join(ownership, ownership.c.owner_id == Owner.id).join(Pet, ownership.c.pet_id == Pet.id) 
with Session(engine) as session:
    print(session.execute(text(raw)).mappings().all())

[{'owner_name': 'nima', 'pet_name': 'asal'}, {'owner_name': 'nima', 'pet_name': 'bambi'}, {'owner_name': 'banafsheh', 'pet_name': 'asal'}]


## left join

In [6]:
from sqlalchemy import inspect
inspect(engine).get_table_names()

['users_file',
 'constraint_core',
 'pet',
 'owner',
 'new_owner',
 'ownership',
 'new_pet',
 'owner3',
 'ownership3',
 'pet3',
 'core_table',
 'new_table',
 'new_table_2',
 'enum_table',
 'orm_enum',
 'orm_enum_2',
 'declaritive_table',
 'registery_table',
 'core_orm',
 'tddl',
 'first',
 'second',
 'tdate_orm',
 'tdate_text',
 'constraint_orm']

In [10]:
new_owner = Owner(owner_name='ali')

In [11]:
from sqlalchemy.orm import Session
with Session(engine) as session:
    session.add(new_owner)
    session.commit()

In [13]:
from sqlalchemy import select
stmt = select(Owner.owner_name, Pet.pet_name).join(Pet, Owner.pet_id == Pet.id)
with Session(engine) as session:
    print(session.execute(stmt).mappings().all())

[{'owner_name': 'nima', 'pet_name': 'asal'}, {'owner_name': 'banafsheh', 'pet_name': 'asal'}]


## text

In [15]:
raw_sql= """
SELECT owner.owner_name, pet.pet_name
FROM owner
LEFT JOIN pet ON owner.pet_id = pet.id
"""

from sqlalchemy import text
with Session(engine) as session:
    print(session.execute(text(raw_sql)).mappings().all())

[{'owner_name': 'nima', 'pet_name': 'asal'}, {'owner_name': 'banafsheh', 'pet_name': 'asal'}, {'owner_name': 'ali', 'pet_name': None}]


## core / orm

In [16]:
stmt = select(Owner.owner_name, Pet.pet_name).outerjoin(Pet, Owner.pet_id == Pet.id)
with Session(engine) as session:
    print(session.execute(stmt).mappings().all())

[{'owner_name': 'nima', 'pet_name': 'asal'}, {'owner_name': 'banafsheh', 'pet_name': 'asal'}, {'owner_name': 'ali', 'pet_name': None}]


# delete when we have foreign key

In [9]:
from sqlalchemy import update

bambi = Pet(pet_name = 'bambi')
update_stmt = update(Owner).where(Owner.id == 3).values(pet_id = 2)
with Session(engine) as session:
    session.add(bambi)
    session.flush()
    session.execute(update_stmt)
    session.commit()

In [12]:
# from sqlalchemy import delete
# from sqlalchemy.orm import Session

# del_stmt = delete(Pet).where(Pet.id == 2)
# with Session(engine) as session:
#     session.execute(del_stmt)
#     session.commit()

In [14]:
update_stmt = update(Owner).where(Owner.id == 3).values(pet_id = None)
del_stmt = delete(Pet).where(Pet.id == 2)
with Session(engine) as session:
    session.execute(update_stmt)
    session.flush()
    session.execute(del_stmt)
    session.commit()

# Use of ORM: session.get and session.delete

### ✅ ORM vs Core — Functional Mapping in SQLAlchemy

| **Operation**            | **SQLAlchemy Core**                                 | **SQLAlchemy ORM**                                          |
|--------------------------|-----------------------------------------------------|--------------------------------------------------------------|
| **Insert**               | `insert(User).values(...)`                          | `session.add(user_obj)`                                     |
| **Bulk Insert**          | `conn.execute(insert(User), data_list)`             | `session.add_all([user1, user2])`                           |
| **Select (by id)**       | `select(User).where(User.id == 1)`                  | `session.get(User, 1)`                                      |
| **Select (filtered)**    | `select(User).where(User.name == "Nima")`           | `session.query(User).filter_by(name="Nima").all()`          |
| **Update**               | `update(User).where(...).values(name="Nima")`       | `user.name = "Nima"` then `session.commit()`                |
| **Delete**               | `delete(User).where(User.id == 1)`                  | `session.delete(user_obj)`                                  |
| **Join**                 | `select().join(Pet, Pet.owner_id == User.id)`       | `user.pets` (if relationship is defined)                    |
| **Add to relationship**  | —                                                   | `user.pets.append(pet_obj)`                                 |
| **Remove from relation** | —                                                   | `user.pets.remove(pet_obj)`                                 |
| **Insert via relation**  | —                                                   | `user.pets = [Pet(...), Pet(...)]`                          |
| **Delete via relation**  | —                                                   | `del user.pets[0]` or `user.pets.clear()`                   |
| **Raw SQL**              | `conn.execute(text("SELECT * FROM users"))`         | `session.execute(text("SELECT * FROM users"))`              |


In [4]:
from sqlalchemy.orm import Session

with Session(engine) as session:
    new_pet = Pet(pet_name= 'asalasal')
    nima = session.get(Owner, 1)
    nima.pet = new_pet
    session.flush()
    old_ver = session.get(Pet, 1)
    session.delete(old_ver)
    session.commit()

In [5]:
with Session(engine) as session:
    nima = session.get(Owner, 1)
    pet = nima.pet

In [6]:
pet

a pet named asalasal

# SERIAL and sequence function

In [3]:
from sqlalchemy import text, select
from sqlalchemy.orm import Session

## checing a sequence

In [6]:
raw_sql = """
SELECT *
from pet_id_seq
"""

with Session(engine) as session:
    print(session.execute(text(raw_sql)).mappings().all())

[{'last_value': 4, 'log_cnt': 32, 'is_called': True}]


## manually calling seq func

In [11]:
raw_sql = """
SELECT
nextval('pet_id_seq'::regclass)
"""

with Session(engine) as session:
    print(session.execute(text(raw_sql)).mappings().all())

## restarting seq

In [16]:
raw_sql = """
ALTER SEQUENCE pet_id_seq
RESTART WITH 4
"""

with Session(engine) as session:
    session.execute(text(raw_sql))
    session.commit()




raw_sql = """
SELECT *
from pet_id_seq
"""

with Session(engine) as session:
    print(session.execute(text(raw_sql)).mappings().all())

[{'last_value': 4, 'log_cnt': 0, 'is_called': False}]


# handle from here

# can we handle seq with orm

# session.begin()

# foreign key constraints (ON DELETE SET NULL)

# sqlaodegen

# pg_constraint

# window function